In [1]:
import numpy as np
import os
import re
import pandas as pd
import pickle
from sklearn.externals import joblib
from keras.models import load_model
import operator

c:\users\guswl\appdata\local\programs\python\python37\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.


# Trainset

In [2]:
file_lst = os.listdir('./')
file_list_npy = [f for f in file_lst if f.endswith(".npy")]
data = {}
for file in file_list_npy :
    #print(file)
    TEMPLATE = np.load('./{0}'.format(file),allow_pickle=True)
    lst = []
    for x,y in TEMPLATE:
        lst.append(x)
        lst.append(y)
    filename = file.split('_')[1]
    filename = filename.split('.')[0]
    data[filename] = lst

In [3]:
df = pd.DataFrame(data)
df = df.transpose()
df.rename(columns = lambda x: 'x'+str(int(x/2)+1) if(x%2 == 0) else 'y'+str(int(x/2)+1), inplace = True)
df = df.sort_index()

In [4]:
dataset = df.loc[:, ['x1','y1','x17','y17','x18','y18','x19','y19','x20','y20','x21','y21','x22','y22','x23','y23',
                     'x24','y24','x25','y25','x26','y26','x27','y27','x28','y28','x37','y37','x38','y38','x39','y39',
                     'x40','y40','x41','y41','x42','y42','x43','y43','x44','y44','x45','y45','x46','y46','x47','y47',
                     'x48','y48']]
dat = df.loc[:, ['x1','y1','x37','y37','x38','y38','x39','y39','x40','y40','x41','y41','x42','y42','x43','y43','x44','y44',
                 'x45','y45','x46','y46','x47','y47','x48','y48','x17','y17']]
dat.rename(columns = {"x37":"x2", "y37":"y2", "x40":"x4", "y40":"y4", "x43":"x5", "y43":"y5",
                      "x46":"x7", "y46":"y7", "x17":"x8","y17":"y8"}, inplace=True)
dat['x3'] = (dat['x38'] + dat['x39'] + dat['x41'] + dat['x42'])/4
dat['y3'] = (dat['y38'] + dat['y39'] + dat['y41'] + dat['y42'])/4
dat['x6'] = (dat['x44'] + dat['x45'] + dat['x47'] + dat['x48'])/4
dat['y6'] = (dat['y44'] + dat['y45'] + dat['y47'] + dat['y48'])/4
dat = dat.loc[:, ['x1','y1','x2','y2','x3','y3','x4','y4','x5','y5','x6','y6','x7','y7','x8','y8']]
dat['1'] = ((dat['x2'] - dat['x1'])**2 + (dat['y2'] - dat['y1'])**2)**1/2
dat['2'] = (dat['y2'] - dat['y1'])/(dat['x2'] - dat['x1'])
dat['3'] = ((dat['x3'] - dat['x2'])**2 + (dat['y3'] - dat['y2'])**2)**1/2
dat['4'] = (dat['y3'] - dat['y2'])/(dat['x3'] - dat['x2'])
dat['5'] = ((dat['x4'] - dat['x3'])**2 + (dat['y4'] - dat['y3'])**2)**1/2
dat['6'] = (dat['y4'] - dat['y3'])/(dat['x4'] - dat['x3'])
dat['7'] = ((dat['x5'] - dat['x4'])**2 + (dat['y5'] - dat['y4'])**2)**1/2
dat['8'] = (dat['y5'] - dat['y4'])/(dat['x5'] - dat['x4'])
dat['9'] = ((dat['x6'] - dat['x5'])**2 + (dat['y6'] - dat['y5'])**2)**1/2
dat['10'] = (dat['y6'] - dat['y5'])/(dat['x6'] - dat['x5'])
dat['11'] = ((dat['x7'] - dat['x6'])**2 + (dat['y7'] - dat['y6'])**2)**1/2
dat['12'] = (dat['y7'] - dat['y6'])/(dat['x7'] - dat['x6'])
dat['13'] = ((dat['x8'] - dat['x7'])**2 + (dat['y8'] - dat['y7'])**2)**1/2
dat['14'] = (dat['y8'] - dat['y7'])/(dat['x8'] - dat['x7'])
dat['15'] = ((dat['x8'] - dat['x1'])**2 + (dat['y8'] - dat['y1'])**2)**1/2
dat['16'] = (dat['y8'] - dat['y1'])/(dat['x8'] - dat['x1'])
dat['17'] = ((dat['x6'] - dat['x3'])**2 + (dat['y6'] - dat['y3'])**2)**1/2
dat['18'] = (dat['y6'] - dat['y3'])/(dat['x6'] - dat['x3'])
data1 = dat.loc[:,'1':'18']
data1 = data1.sort_index()

In [5]:
train_label = list(data1.index)
for i in range(len(train_label)) :
    ans = re.findall("\d+", train_label[i])[0]
    train_label[i] = int(ans)-1

# Testset

In [6]:
file_lst = os.listdir('./test/')
file_list_npy = [f for f in file_lst if f.endswith(".npy")]
data = {}
for file in file_list_npy :
    #print(file)
    TEMPLATE = np.load('./test/{0}'.format(file),allow_pickle=True)
    lst = []
    for x,y in TEMPLATE:
        lst.append(x)
        lst.append(y)
    filename = file.split('_')[1]
    filename = filename.split('.')[0]
    data[filename] = lst

In [7]:
df2 = pd.DataFrame(data)
df2 = df2.transpose()
df2.rename(columns = lambda x: 'x'+str(int(x/2)+1) if(x%2 == 0) else 'y'+str(int(x/2)+1), inplace = True)
df2 = df2.sort_index()

In [8]:
dataset2 = df2.loc[:, ['x1','y1','x17','y17','x18','y18','x19','y19','x20','y20','x21','y21','x22','y22','x23','y23',
                     'x24','y24','x25','y25','x26','y26','x27','y27','x28','y28','x37','y37','x38','y38','x39','y39',
                     'x40','y40','x41','y41','x42','y42','x43','y43','x44','y44','x45','y45','x46','y46','x47','y47',
                     'x48','y48']]
dat2 = df2.loc[:, ['x1','y1','x37','y37','x38','y38','x39','y39','x40','y40','x41','y41','x42','y42','x43','y43','x44','y44',
                 'x45','y45','x46','y46','x47','y47','x48','y48','x17','y17']]
dat2.rename(columns = {"x37":"x2", "y37":"y2", "x40":"x4", "y40":"y4", "x43":"x5", "y43":"y5",
                      "x46":"x7", "y46":"y7", "x17":"x8","y17":"y8"}, inplace=True)
dat2['x3'] = (dat2['x38'] + dat2['x39'] + dat2['x41'] + dat2['x42'])/4
dat2['y3'] = (dat2['y38'] + dat2['y39'] + dat2['y41'] + dat2['y42'])/4
dat2['x6'] = (dat2['x44'] + dat2['x45'] + dat2['x47'] + dat2['x48'])/4
dat2['y6'] = (dat2['y44'] + dat2['y45'] + dat2['y47'] + dat2['y48'])/4
dat2 = dat2.loc[:, ['x1','y1','x2','y2','x3','y3','x4','y4','x5','y5','x6','y6','x7','y7','x8','y8']]
dat2['1'] = ((dat2['x2'] - dat2['x1'])**2 + (dat2['y2'] - dat2['y1'])**2)**1/2
dat2['2'] = (dat2['y2'] - dat2['y1'])/(dat2['x2'] - dat2['x1'])
dat2['3'] = ((dat2['x3'] - dat2['x2'])**2 + (dat2['y3'] - dat2['y2'])**2)**1/2
dat2['4'] = (dat2['y3'] - dat2['y2'])/(dat2['x3'] - dat2['x2'])
dat2['5'] = ((dat2['x4'] - dat2['x3'])**2 + (dat2['y4'] - dat2['y3'])**2)**1/2
dat2['6'] = (dat2['y4'] - dat2['y3'])/(dat2['x4'] - dat2['x3'])
dat2['7'] = ((dat2['x5'] - dat2['x4'])**2 + (dat2['y5'] - dat2['y4'])**2)**1/2
dat2['8'] = (dat2['y5'] - dat2['y4'])/(dat2['x5'] - dat2['x4'])
dat2['9'] = ((dat2['x6'] - dat2['x5'])**2 + (dat2['y6'] - dat2['y5'])**2)**1/2
dat2['10'] = (dat2['y6'] - dat2['y5'])/(dat2['x6'] - dat2['x5'])
dat2['11'] = ((dat2['x7'] - dat2['x6'])**2 + (dat2['y7'] - dat2['y6'])**2)**1/2
dat2['12'] = (dat2['y7'] - dat2['y6'])/(dat2['x7'] - dat2['x6'])
dat2['13'] = ((dat2['x8'] - dat2['x7'])**2 + (dat2['y8'] - dat2['y7'])**2)**1/2
dat2['14'] = (dat2['y8'] - dat2['y7'])/(dat2['x8'] - dat2['x7'])
dat2['15'] = ((dat2['x8'] - dat2['x1'])**2 + (dat2['y8'] - dat2['y1'])**2)**1/2
dat2['16'] = (dat2['y8'] - dat2['y1'])/(dat2['x8'] - dat2['x1'])
dat2['17'] = ((dat2['x6'] - dat2['x3'])**2 + (dat2['y6'] - dat2['y3'])**2)**1/2
dat2['18'] = (dat2['y6'] - dat2['y3'])/(dat2['x6'] - dat2['x3'])
data2 = dat2.loc[:,'1':'18']
data2 = data2.sort_index()

In [9]:
test_label = list(data2.index)
for i in range(len(test_label)) :
    ans = re.findall("\d+", test_label[i])[0]
    test_label[i] = (int(ans)-1)%10

# Embedding

In [10]:
emb_train = pd.read_csv("./reps.csv", header=None)
emb_train.rename(columns = lambda x: str(int(x)+1), inplace = True)

In [11]:
emb_test = pd.read_csv("./testset/reps.csv", header=None)
emb_test.rename(columns = lambda x: str(int(x)+1), inplace = True)

In [12]:
label_train = pd.read_csv("./labels.csv", header=None)
label_train = list(label_train[1])
train = []
idx_train = []
for i in range(len(label_train)):
    train.append(i)
    idx_train.append(label_train[i].split('/')[3].split('_')[0])

In [13]:
label_test = pd.read_csv("./testset/labels.csv", header=None)
label_test = list(label_test[1])
test = []
idx_test = []
for i in range(len(label_test)):
    if '-' not in (label_test[i].split('/')[3]):
        test.append(i)
        idx_test.append(label_test[i].split('/')[3].split('_')[0])

In [14]:
emb_train = emb_train.iloc[train]
emb_train['idx'] = idx_train
emb_train = emb_train.set_index("idx")
emb_train = emb_train.sort_index()

In [15]:
emb_test = emb_test.iloc[test]
emb_test['idx'] = idx_test
emb_test = emb_test.set_index("idx")
emb_test = emb_test.sort_index()

In [16]:
train1 = np.array(data1.loc[:, '1':'18'])
train2 = np.array(df.loc[:, 'x1':'y68'])
train34 = np.array(dataset.loc[:, 'x1':'y48'])
train56 = np.array(emb_train.loc[:, '1':'128'])
print(train1.shape, train2.shape, train34.shape, train56.shape)

(500, 18) (500, 136) (500, 50) (500, 128)


In [17]:
test1 = np.array(data2.loc[:, '1':'18'])
test2 = np.array(df2.loc[:, 'x1':'y68'])
test34 = np.array(dataset2.loc[:, 'x1':'y48'])
test56 = np.array(emb_test.loc[:, '1':'128'])
print(test1.shape, test2.shape, test34.shape, test56.shape)

(10, 18) (10, 136) (10, 50) (10, 128)


In [18]:
from keras.utils import np_utils

train_label = np.array(train_label)
test_label = np.array(test_label)
train_y = np_utils.to_categorical(train_label)
test_y = np_utils.to_categorical(test_label)
print(train_y.shape, test_y.shape)

(500, 10) (10, 10)


# 방법 1, Distance and Slope

In [19]:
def MLP_model() :
    # create model
    model = Sequential()
    model.add(Dense(10, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint


# build the model
model = MLP_model()
mc = ModelCheckpoint('best_1model.h5', monitor='val_accuracy', mode='max', save_best_only=True)
# Fit the model
hist = model.fit(train1, train_y, validation_data=(test1, test_y), nb_epoch=5000, batch_size=200, verbose=2, callbacks=[mc])
# Final evaluation of the model
scores = model.evaluate(test1, test_y, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

c:\users\guswl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, activation="softmax", kernel_initializer="normal")`
  after removing the cwd from sys.path.
c:\users\guswl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # Remove the CWD from sys.path while we load stuff.


Train on 500 samples, validate on 10 samples
Epoch 1/5000
 - 0s - loss: 371.8260 - accuracy: 0.1000 - val_loss: 339.1649 - val_accuracy: 0.1000
Epoch 2/5000
 - 0s - loss: 339.1317 - accuracy: 0.1000 - val_loss: 307.3619 - val_accuracy: 0.1000
Epoch 3/5000
 - 0s - loss: 306.6279 - accuracy: 0.0960 - val_loss: 277.5508 - val_accuracy: 0.1000
Epoch 4/5000
 - 0s - loss: 277.0520 - accuracy: 0.0900 - val_loss: 257.4860 - val_accuracy: 0.1000
Epoch 5/5000
 - 0s - loss: 256.0334 - accuracy: 0.1000 - val_loss: 232.4258 - val_accuracy: 0.1000
Epoch 6/5000
 - 0s - loss: 229.0901 - accuracy: 0.1020 - val_loss: 204.8785 - val_accuracy: 0.1000
Epoch 7/5000
 - 0s - loss: 203.2056 - accuracy: 0.1040 - val_loss: 188.7197 - val_accuracy: 0.0000e+00
Epoch 8/5000
 - 0s - loss: 187.1293 - accuracy: 0.0940 - val_loss: 170.0723 - val_accuracy: 0.0000e+00
Epoch 9/5000
 - 0s - loss: 166.5685 - accuracy: 0.0780 - val_loss: 147.6638 - val_accuracy: 0.0000e+00
Epoch 10/5000
 - 0s - loss: 143.5018 - accuracy: 0.0

Epoch 83/5000
 - 0s - loss: 3.5772 - accuracy: 0.0940 - val_loss: 5.7487 - val_accuracy: 0.0000e+00
Epoch 84/5000
 - 0s - loss: 3.5833 - accuracy: 0.1120 - val_loss: 5.4733 - val_accuracy: 0.0000e+00
Epoch 85/5000
 - 0s - loss: 3.5284 - accuracy: 0.0800 - val_loss: 5.5585 - val_accuracy: 0.0000e+00
Epoch 86/5000
 - 0s - loss: 3.4874 - accuracy: 0.0680 - val_loss: 5.5102 - val_accuracy: 0.0000e+00
Epoch 87/5000
 - 0s - loss: 3.5129 - accuracy: 0.1300 - val_loss: 5.4466 - val_accuracy: 0.0000e+00
Epoch 88/5000
 - 0s - loss: 3.4620 - accuracy: 0.0920 - val_loss: 5.3895 - val_accuracy: 0.1000
Epoch 89/5000
 - 0s - loss: 3.4517 - accuracy: 0.1000 - val_loss: 5.2519 - val_accuracy: 0.0000e+00
Epoch 90/5000
 - 0s - loss: 3.4049 - accuracy: 0.1040 - val_loss: 5.2864 - val_accuracy: 0.0000e+00
Epoch 91/5000
 - 0s - loss: 3.4222 - accuracy: 0.1000 - val_loss: 5.0752 - val_accuracy: 0.0000e+00
Epoch 92/5000
 - 0s - loss: 3.4114 - accuracy: 0.1180 - val_loss: 5.2533 - val_accuracy: 0.1000
Epoch 93

Epoch 166/5000
 - 0s - loss: 2.5177 - accuracy: 0.1460 - val_loss: 3.1920 - val_accuracy: 0.1000
Epoch 167/5000
 - 0s - loss: 2.5359 - accuracy: 0.1520 - val_loss: 3.2499 - val_accuracy: 0.1000
Epoch 168/5000
 - 0s - loss: 2.5178 - accuracy: 0.1420 - val_loss: 3.3125 - val_accuracy: 0.2000
Epoch 169/5000
 - 0s - loss: 2.5278 - accuracy: 0.1660 - val_loss: 3.1554 - val_accuracy: 0.0000e+00
Epoch 170/5000
 - 0s - loss: 2.5113 - accuracy: 0.1360 - val_loss: 3.2346 - val_accuracy: 0.1000
Epoch 171/5000
 - 0s - loss: 2.4707 - accuracy: 0.1720 - val_loss: 3.1605 - val_accuracy: 0.1000
Epoch 172/5000
 - 0s - loss: 2.5071 - accuracy: 0.1320 - val_loss: 3.1551 - val_accuracy: 0.1000
Epoch 173/5000
 - 0s - loss: 2.4493 - accuracy: 0.1860 - val_loss: 3.2588 - val_accuracy: 0.1000
Epoch 174/5000
 - 0s - loss: 2.4145 - accuracy: 0.1520 - val_loss: 3.1990 - val_accuracy: 0.1000
Epoch 175/5000
 - 0s - loss: 2.4128 - accuracy: 0.1640 - val_loss: 3.1921 - val_accuracy: 0.2000
Epoch 176/5000
 - 0s - los

Epoch 251/5000
 - 0s - loss: 2.1243 - accuracy: 0.2540 - val_loss: 2.9078 - val_accuracy: 0.1000
Epoch 252/5000
 - 0s - loss: 2.0910 - accuracy: 0.2600 - val_loss: 2.7755 - val_accuracy: 0.0000e+00
Epoch 253/5000
 - 0s - loss: 2.0798 - accuracy: 0.2420 - val_loss: 2.9004 - val_accuracy: 0.2000
Epoch 254/5000
 - 0s - loss: 2.1279 - accuracy: 0.2760 - val_loss: 2.7627 - val_accuracy: 0.1000
Epoch 255/5000
 - 0s - loss: 2.0980 - accuracy: 0.2420 - val_loss: 2.7956 - val_accuracy: 0.2000
Epoch 256/5000
 - 0s - loss: 2.0841 - accuracy: 0.2700 - val_loss: 2.8091 - val_accuracy: 0.1000
Epoch 257/5000
 - 0s - loss: 2.0229 - accuracy: 0.2700 - val_loss: 2.7887 - val_accuracy: 0.2000
Epoch 258/5000
 - 0s - loss: 2.0425 - accuracy: 0.2920 - val_loss: 2.7775 - val_accuracy: 0.1000
Epoch 259/5000
 - 0s - loss: 2.0261 - accuracy: 0.2720 - val_loss: 2.7964 - val_accuracy: 0.1000
Epoch 260/5000
 - 0s - loss: 2.0014 - accuracy: 0.3080 - val_loss: 2.7637 - val_accuracy: 0.1000
Epoch 261/5000
 - 0s - los

...

Epoch 4622/5000
 - 0s - loss: 1.4594 - accuracy: 0.4740 - val_loss: 2.9575 - val_accuracy: 0.2000
Epoch 4623/5000
 - 0s - loss: 1.4583 - accuracy: 0.4700 - val_loss: 2.9289 - val_accuracy: 0.2000
Epoch 4624/5000
 - 0s - loss: 1.5126 - accuracy: 0.4400 - val_loss: 2.9112 - val_accuracy: 0.2000
Epoch 4625/5000
 - 0s - loss: 1.4578 - accuracy: 0.4700 - val_loss: 2.8151 - val_accuracy: 0.2000
Epoch 4626/5000
 - 0s - loss: 1.5032 - accuracy: 0.4640 - val_loss: 2.8808 - val_accuracy: 0.3000
Epoch 4627/5000
 - 0s - loss: 1.4574 - accuracy: 0.4700 - val_loss: 2.8673 - val_accuracy: 0.1000
Epoch 4628/5000
 - 0s - loss: 1.4725 - accuracy: 0.4660 - val_loss: 2.9181 - val_accuracy: 0.2000
Epoch 4629/5000
 - 0s - loss: 1.4589 - accuracy: 0.4900 - val_loss: 2.8369 - val_accuracy: 0.3000
Epoch 4630/5000
 - 0s - loss: 1.4743 - accuracy: 0.4860 - val_loss: 2.8735 - val_accuracy: 0.2000
Epoch 4631/5000
 - 0s - loss: 1.4894 - accuracy: 0.4680 - val_loss: 2.8380 - val_accuracy: 0.3000
Epoch 4632/5000
 - 0

Epoch 4706/5000
 - 0s - loss: 1.4944 - accuracy: 0.4520 - val_loss: 2.9243 - val_accuracy: 0.1000
Epoch 4707/5000
 - 0s - loss: 1.4477 - accuracy: 0.4860 - val_loss: 2.8893 - val_accuracy: 0.2000
Epoch 4708/5000
 - 0s - loss: 1.3881 - accuracy: 0.5080 - val_loss: 2.8020 - val_accuracy: 0.0000e+00
Epoch 4709/5000
 - 0s - loss: 1.4036 - accuracy: 0.5200 - val_loss: 2.8519 - val_accuracy: 0.3000
Epoch 4710/5000
 - 0s - loss: 1.3619 - accuracy: 0.5260 - val_loss: 2.8961 - val_accuracy: 0.2000
Epoch 4711/5000
 - 0s - loss: 1.3906 - accuracy: 0.4680 - val_loss: 2.7950 - val_accuracy: 0.3000
Epoch 4712/5000
 - 0s - loss: 1.3672 - accuracy: 0.5020 - val_loss: 2.8489 - val_accuracy: 0.1000
Epoch 4713/5000
 - 0s - loss: 1.3543 - accuracy: 0.5000 - val_loss: 2.8175 - val_accuracy: 0.3000
Epoch 4714/5000
 - 0s - loss: 1.3427 - accuracy: 0.5220 - val_loss: 2.9188 - val_accuracy: 0.1000
Epoch 4715/5000
 - 0s - loss: 1.3811 - accuracy: 0.4980 - val_loss: 2.8508 - val_accuracy: 0.2000
Epoch 4716/5000


Epoch 4790/5000
 - 0s - loss: 1.4094 - accuracy: 0.5040 - val_loss: 2.8290 - val_accuracy: 0.2000
Epoch 4791/5000
 - 0s - loss: 1.4128 - accuracy: 0.5060 - val_loss: 2.8957 - val_accuracy: 0.3000
Epoch 4792/5000
 - 0s - loss: 1.3866 - accuracy: 0.5020 - val_loss: 2.7857 - val_accuracy: 0.0000e+00
Epoch 4793/5000
 - 0s - loss: 1.3971 - accuracy: 0.5100 - val_loss: 2.8449 - val_accuracy: 0.1000
Epoch 4794/5000
 - 0s - loss: 1.3624 - accuracy: 0.5200 - val_loss: 2.8437 - val_accuracy: 0.1000
Epoch 4795/5000
 - 0s - loss: 1.4184 - accuracy: 0.4900 - val_loss: 2.7754 - val_accuracy: 0.2000
Epoch 4796/5000
 - 0s - loss: 1.3596 - accuracy: 0.5320 - val_loss: 2.8585 - val_accuracy: 0.1000
Epoch 4797/5000
 - 0s - loss: 1.3943 - accuracy: 0.4780 - val_loss: 2.8460 - val_accuracy: 0.2000
Epoch 4798/5000
 - 0s - loss: 1.4242 - accuracy: 0.4980 - val_loss: 2.8479 - val_accuracy: 0.2000
Epoch 4799/5000
 - 0s - loss: 1.3580 - accuracy: 0.5020 - val_loss: 2.8014 - val_accuracy: 0.1000
Epoch 4800/5000


Epoch 4874/5000
 - 0s - loss: 1.5027 - accuracy: 0.5080 - val_loss: 2.7808 - val_accuracy: 0.0000e+00
Epoch 4875/5000
 - 0s - loss: 1.4333 - accuracy: 0.4720 - val_loss: 2.7988 - val_accuracy: 0.2000
Epoch 4876/5000
 - 0s - loss: 1.4621 - accuracy: 0.5000 - val_loss: 2.8246 - val_accuracy: 0.1000
Epoch 4877/5000
 - 0s - loss: 1.4937 - accuracy: 0.4680 - val_loss: 2.8422 - val_accuracy: 0.1000
Epoch 4878/5000
 - 0s - loss: 1.4650 - accuracy: 0.4840 - val_loss: 2.8541 - val_accuracy: 0.2000
Epoch 4879/5000
 - 0s - loss: 1.5008 - accuracy: 0.4440 - val_loss: 2.8597 - val_accuracy: 0.1000
Epoch 4880/5000
 - 0s - loss: 1.4260 - accuracy: 0.5000 - val_loss: 2.9932 - val_accuracy: 0.1000
Epoch 4881/5000
 - 0s - loss: 1.4583 - accuracy: 0.4880 - val_loss: 2.8325 - val_accuracy: 0.2000
Epoch 4882/5000
 - 0s - loss: 1.4099 - accuracy: 0.4880 - val_loss: 2.9084 - val_accuracy: 0.2000
Epoch 4883/5000
 - 0s - loss: 1.3845 - accuracy: 0.4760 - val_loss: 2.8148 - val_accuracy: 0.2000
Epoch 4884/5000


Epoch 4958/5000
 - 0s - loss: 1.4379 - accuracy: 0.4740 - val_loss: 2.7962 - val_accuracy: 0.2000
Epoch 4959/5000
 - 0s - loss: 1.4877 - accuracy: 0.4540 - val_loss: 2.9504 - val_accuracy: 0.3000
Epoch 4960/5000
 - 0s - loss: 1.4515 - accuracy: 0.4900 - val_loss: 2.8309 - val_accuracy: 0.2000
Epoch 4961/5000
 - 0s - loss: 1.4889 - accuracy: 0.4260 - val_loss: 2.9555 - val_accuracy: 0.2000
Epoch 4962/5000
 - 0s - loss: 1.4804 - accuracy: 0.4900 - val_loss: 2.8523 - val_accuracy: 0.2000
Epoch 4963/5000
 - 0s - loss: 1.4625 - accuracy: 0.4560 - val_loss: 2.9486 - val_accuracy: 0.3000
Epoch 4964/5000
 - 0s - loss: 1.4920 - accuracy: 0.4660 - val_loss: 2.8699 - val_accuracy: 0.0000e+00
Epoch 4965/5000
 - 0s - loss: 1.4071 - accuracy: 0.5220 - val_loss: 2.8206 - val_accuracy: 0.1000
Epoch 4966/5000
 - 0s - loss: 1.4082 - accuracy: 0.4920 - val_loss: 2.8540 - val_accuracy: 0.1000
Epoch 4967/5000
 - 0s - loss: 1.3550 - accuracy: 0.5060 - val_loss: 2.8790 - val_accuracy: 0.3000
Epoch 4968/5000


In [21]:
max(hist.history['val_accuracy'])

0.4000000059604645

In [22]:
from keras.models import load_model

b_model = load_model('best_1model.h5')
#b_model = load_model('./10testset_best_model/best_1model.h5')

answer = []
pred = []
answer_per = []
predict_per = []

for i in range(10) :
    test = test1[i].reshape(1,18)
    label = np.argmax(test_y[i])
    predict = b_model.predict(test)
    max_val = np.argmax(predict[0])
    answer.append(label)
    pred.append(max_val)
    answer_per.append(predict[0][label])
    predict_per.append(predict[0])
    print(label, max_val, predict[0][label], predict[0][max_val])

0 9 0.046929304 0.70776546
9 9 0.4274634 0.4274634
1 1 0.36310625 0.36310625
2 2 0.2707056 0.2707056
3 8 0.059744097 0.447748
4 9 0.10460911 0.5823509
5 2 0.023513282 0.63213307
6 9 0.033075076 0.27810296
7 7 0.2034512 0.2034512
8 9 8.340216e-05 0.5573029


# 방법 2. 68 Landmark

In [23]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint

# build the model
model = MLP_model()
mc = ModelCheckpoint('best_2model.h5', monitor='val_accuracy', mode='max', save_best_only=True)
# Fit the model
hist = model.fit(train2, train_y, validation_data=(test2, test_y), nb_epoch=5000, batch_size=200, verbose=2, callbacks=[mc])
# Final evaluation of the model
scores = model.evaluate(test2, test_y, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

c:\users\guswl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, activation="softmax", kernel_initializer="normal")`
  after removing the cwd from sys.path.
c:\users\guswl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:9: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Train on 500 samples, validate on 10 samples
Epoch 1/5000
 - 0s - loss: 30.5239 - accuracy: 0.1040 - val_loss: 18.2274 - val_accuracy: 0.1000
Epoch 2/5000
 - 0s - loss: 16.1786 - accuracy: 0.0680 - val_loss: 10.2962 - val_accuracy: 0.1000
Epoch 3/5000
 - 0s - loss: 10.3193 - accuracy: 0.0740 - val_loss: 10.0701 - val_accuracy: 0.2000
Epoch 4/5000
 - 0s - loss: 11.0670 - accuracy: 0.0900 - val_loss: 9.9391 - val_accuracy: 0.1000
Epoch 5/5000
 - 0s - loss: 9.3365 - accuracy: 0.0960 - val_loss: 8.0960 - val_accuracy: 0.0000e+00
Epoch 6/5000
 - 0s - loss: 8.2429 - accuracy: 0.0820 - val_loss: 8.6025 - val_accuracy: 0.1000
Epoch 7/5000
 - 0s - loss: 7.5792 - accuracy: 0.0780 - val_loss: 6.2644 - val_accuracy: 0.2000
Epoch 8/5000
 - 0s - loss: 5.9061 - accuracy: 0.0820 - val_loss: 5.4462 - val_accuracy: 0.1000
Epoch 9/5000
 - 0s - loss: 5.2935 - accuracy: 0.1000 - val_loss: 3.9616 - val_accuracy: 0.1000
Epoch 10/5000
 - 0s - loss: 4.5237 - accuracy: 0.1040 - val_loss: 4.8158 - val_accuracy: 

Epoch 86/5000
 - 0s - loss: 2.0908 - accuracy: 0.2720 - val_loss: 3.0111 - val_accuracy: 0.0000e+00
Epoch 87/5000
 - 0s - loss: 2.1227 - accuracy: 0.2660 - val_loss: 3.1249 - val_accuracy: 0.1000
Epoch 88/5000
 - 0s - loss: 2.0653 - accuracy: 0.2680 - val_loss: 3.1263 - val_accuracy: 0.1000
Epoch 89/5000
 - 0s - loss: 2.0553 - accuracy: 0.3000 - val_loss: 2.9555 - val_accuracy: 0.2000
Epoch 90/5000
 - 0s - loss: 2.0540 - accuracy: 0.2800 - val_loss: 3.0989 - val_accuracy: 0.1000
Epoch 91/5000
 - 0s - loss: 2.0742 - accuracy: 0.2960 - val_loss: 3.0105 - val_accuracy: 0.1000
Epoch 92/5000
 - 0s - loss: 2.0399 - accuracy: 0.3060 - val_loss: 3.0919 - val_accuracy: 0.1000
Epoch 93/5000
 - 0s - loss: 2.0241 - accuracy: 0.2940 - val_loss: 3.0906 - val_accuracy: 0.1000
Epoch 94/5000
 - 0s - loss: 1.9884 - accuracy: 0.3260 - val_loss: 3.0024 - val_accuracy: 0.1000
Epoch 95/5000
 - 0s - loss: 2.0063 - accuracy: 0.3260 - val_loss: 3.1082 - val_accuracy: 0.1000
Epoch 96/5000
 - 0s - loss: 1.9993 -

Epoch 171/5000
 - 0s - loss: 1.7693 - accuracy: 0.3880 - val_loss: 2.7518 - val_accuracy: 0.1000
Epoch 172/5000
 - 0s - loss: 1.7056 - accuracy: 0.4660 - val_loss: 2.8205 - val_accuracy: 0.1000
Epoch 173/5000
 - 0s - loss: 1.6481 - accuracy: 0.4760 - val_loss: 2.8647 - val_accuracy: 0.0000e+00
Epoch 174/5000
 - 0s - loss: 1.7359 - accuracy: 0.4120 - val_loss: 2.9045 - val_accuracy: 0.1000
Epoch 175/5000
 - 0s - loss: 1.7317 - accuracy: 0.4640 - val_loss: 2.6340 - val_accuracy: 0.1000
Epoch 176/5000
 - 0s - loss: 1.6949 - accuracy: 0.4340 - val_loss: 2.7259 - val_accuracy: 0.1000
Epoch 177/5000
 - 0s - loss: 1.7480 - accuracy: 0.4240 - val_loss: 2.8120 - val_accuracy: 0.2000
Epoch 178/5000
 - 0s - loss: 1.7455 - accuracy: 0.4160 - val_loss: 2.6264 - val_accuracy: 0.1000
Epoch 179/5000
 - 0s - loss: 1.6571 - accuracy: 0.4740 - val_loss: 2.8382 - val_accuracy: 0.1000
Epoch 180/5000
 - 0s - loss: 1.7263 - accuracy: 0.4080 - val_loss: 2.6758 - val_accuracy: 0.0000e+00
Epoch 181/5000
 - 0s -

Epoch 256/5000
 - 0s - loss: 1.3648 - accuracy: 0.5760 - val_loss: 2.5558 - val_accuracy: 0.2000
Epoch 257/5000
 - 0s - loss: 1.3809 - accuracy: 0.5980 - val_loss: 2.4840 - val_accuracy: 0.3000
Epoch 258/5000
 - 0s - loss: 1.3782 - accuracy: 0.6020 - val_loss: 2.3863 - val_accuracy: 0.1000
Epoch 259/5000
 - 0s - loss: 1.3887 - accuracy: 0.5740 - val_loss: 2.5416 - val_accuracy: 0.2000
Epoch 260/5000
 - 0s - loss: 1.3916 - accuracy: 0.5980 - val_loss: 2.4166 - val_accuracy: 0.2000
Epoch 261/5000
 - 0s - loss: 1.3623 - accuracy: 0.5940 - val_loss: 2.5055 - val_accuracy: 0.2000
Epoch 262/5000
 - 0s - loss: 1.3501 - accuracy: 0.5980 - val_loss: 2.4661 - val_accuracy: 0.2000
Epoch 263/5000
 - 0s - loss: 1.3873 - accuracy: 0.5700 - val_loss: 2.4366 - val_accuracy: 0.2000
Epoch 264/5000
 - 0s - loss: 1.3756 - accuracy: 0.5660 - val_loss: 2.5442 - val_accuracy: 0.2000
Epoch 265/5000
 - 0s - loss: 1.3501 - accuracy: 0.6080 - val_loss: 2.3850 - val_accuracy: 0.1000
Epoch 266/5000
 - 0s - loss: 1

...

Epoch 4633/5000
 - 0s - loss: 0.1132 - accuracy: 0.9880 - val_loss: 2.9676 - val_accuracy: 0.4000
Epoch 4634/5000
 - 0s - loss: 0.1164 - accuracy: 0.9860 - val_loss: 2.9101 - val_accuracy: 0.4000
Epoch 4635/5000
 - 0s - loss: 0.1123 - accuracy: 0.9840 - val_loss: 2.8616 - val_accuracy: 0.3000
Epoch 4636/5000
 - 0s - loss: 0.1143 - accuracy: 0.9860 - val_loss: 3.0168 - val_accuracy: 0.4000
Epoch 4637/5000
 - 0s - loss: 0.1084 - accuracy: 0.9940 - val_loss: 2.8826 - val_accuracy: 0.4000
Epoch 4638/5000
 - 0s - loss: 0.1098 - accuracy: 0.9900 - val_loss: 2.8550 - val_accuracy: 0.3000
Epoch 4639/5000
 - 0s - loss: 0.1116 - accuracy: 0.9920 - val_loss: 3.1329 - val_accuracy: 0.4000
Epoch 4640/5000
 - 0s - loss: 0.1194 - accuracy: 0.9840 - val_loss: 2.7628 - val_accuracy: 0.3000
Epoch 4641/5000
 - 0s - loss: 0.1227 - accuracy: 0.9860 - val_loss: 3.0163 - val_accuracy: 0.5000
Epoch 4642/5000
 - 0s - loss: 0.1534 - accuracy: 0.9680 - val_loss: 2.9041 - val_accuracy: 0.3000
Epoch 4643/5000
 - 0

Epoch 4717/5000
 - 0s - loss: 0.1238 - accuracy: 0.9840 - val_loss: 2.9626 - val_accuracy: 0.4000
Epoch 4718/5000
 - 0s - loss: 0.1273 - accuracy: 0.9780 - val_loss: 3.0737 - val_accuracy: 0.4000
Epoch 4719/5000
 - 0s - loss: 0.1332 - accuracy: 0.9820 - val_loss: 3.0446 - val_accuracy: 0.3000
Epoch 4720/5000
 - 0s - loss: 0.1355 - accuracy: 0.9760 - val_loss: 2.9521 - val_accuracy: 0.5000
Epoch 4721/5000
 - 0s - loss: 0.1638 - accuracy: 0.9480 - val_loss: 3.3591 - val_accuracy: 0.2000
Epoch 4722/5000
 - 0s - loss: 0.1620 - accuracy: 0.9680 - val_loss: 3.1103 - val_accuracy: 0.5000
Epoch 4723/5000
 - 0s - loss: 0.1881 - accuracy: 0.9420 - val_loss: 3.0480 - val_accuracy: 0.3000
Epoch 4724/5000
 - 0s - loss: 0.1414 - accuracy: 0.9720 - val_loss: 3.4738 - val_accuracy: 0.5000
Epoch 4725/5000
 - 0s - loss: 0.1432 - accuracy: 0.9740 - val_loss: 2.8885 - val_accuracy: 0.3000
Epoch 4726/5000
 - 0s - loss: 0.1426 - accuracy: 0.9720 - val_loss: 3.1992 - val_accuracy: 0.4000
Epoch 4727/5000
 - 0

Epoch 4801/5000
 - 0s - loss: 0.1126 - accuracy: 0.9840 - val_loss: 2.9005 - val_accuracy: 0.4000
Epoch 4802/5000
 - 0s - loss: 0.1105 - accuracy: 0.9880 - val_loss: 3.1612 - val_accuracy: 0.3000
Epoch 4803/5000
 - 0s - loss: 0.1137 - accuracy: 0.9940 - val_loss: 3.0287 - val_accuracy: 0.4000
Epoch 4804/5000
 - 0s - loss: 0.1116 - accuracy: 0.9860 - val_loss: 2.8890 - val_accuracy: 0.4000
Epoch 4805/5000
 - 0s - loss: 0.1111 - accuracy: 0.9920 - val_loss: 3.0210 - val_accuracy: 0.4000
Epoch 4806/5000
 - 0s - loss: 0.1236 - accuracy: 0.9720 - val_loss: 3.1020 - val_accuracy: 0.3000
Epoch 4807/5000
 - 0s - loss: 0.1271 - accuracy: 0.9820 - val_loss: 2.9190 - val_accuracy: 0.5000
Epoch 4808/5000
 - 0s - loss: 0.1168 - accuracy: 0.9800 - val_loss: 2.8836 - val_accuracy: 0.4000
Epoch 4809/5000
 - 0s - loss: 0.1203 - accuracy: 0.9860 - val_loss: 3.1792 - val_accuracy: 0.5000
Epoch 4810/5000
 - 0s - loss: 0.1350 - accuracy: 0.9700 - val_loss: 2.9629 - val_accuracy: 0.4000
Epoch 4811/5000
 - 0

Epoch 4885/5000
 - 0s - loss: 0.1317 - accuracy: 0.9760 - val_loss: 3.1908 - val_accuracy: 0.3000
Epoch 4886/5000
 - 0s - loss: 0.1281 - accuracy: 0.9820 - val_loss: 3.0232 - val_accuracy: 0.4000
Epoch 4887/5000
 - 0s - loss: 0.1114 - accuracy: 0.9820 - val_loss: 2.9909 - val_accuracy: 0.3000
Epoch 4888/5000
 - 0s - loss: 0.1178 - accuracy: 0.9820 - val_loss: 3.0292 - val_accuracy: 0.4000
Epoch 4889/5000
 - 0s - loss: 0.1169 - accuracy: 0.9880 - val_loss: 3.2303 - val_accuracy: 0.4000
Epoch 4890/5000
 - 0s - loss: 0.1191 - accuracy: 0.9800 - val_loss: 2.9494 - val_accuracy: 0.4000
Epoch 4891/5000
 - 0s - loss: 0.1247 - accuracy: 0.9840 - val_loss: 3.1017 - val_accuracy: 0.5000
Epoch 4892/5000
 - 0s - loss: 0.1195 - accuracy: 0.9840 - val_loss: 3.0353 - val_accuracy: 0.4000
Epoch 4893/5000
 - 0s - loss: 0.1121 - accuracy: 0.9900 - val_loss: 2.9942 - val_accuracy: 0.4000
Epoch 4894/5000
 - 0s - loss: 0.1153 - accuracy: 0.9680 - val_loss: 2.9581 - val_accuracy: 0.3000
Epoch 4895/5000
 - 0

Epoch 4969/5000
 - 0s - loss: 0.1101 - accuracy: 0.9820 - val_loss: 3.1090 - val_accuracy: 0.4000
Epoch 4970/5000
 - 0s - loss: 0.1050 - accuracy: 0.9880 - val_loss: 2.9258 - val_accuracy: 0.3000
Epoch 4971/5000
 - 0s - loss: 0.0960 - accuracy: 0.9940 - val_loss: 3.2943 - val_accuracy: 0.4000
Epoch 4972/5000
 - 0s - loss: 0.0990 - accuracy: 0.9920 - val_loss: 2.9950 - val_accuracy: 0.3000
Epoch 4973/5000
 - 0s - loss: 0.1100 - accuracy: 0.9840 - val_loss: 2.9832 - val_accuracy: 0.4000
Epoch 4974/5000
 - 0s - loss: 0.1078 - accuracy: 0.9840 - val_loss: 3.0626 - val_accuracy: 0.3000
Epoch 4975/5000
 - 0s - loss: 0.1235 - accuracy: 0.9860 - val_loss: 3.3814 - val_accuracy: 0.5000
Epoch 4976/5000
 - 0s - loss: 0.1294 - accuracy: 0.9760 - val_loss: 2.8875 - val_accuracy: 0.3000
Epoch 4977/5000
 - 0s - loss: 0.1422 - accuracy: 0.9680 - val_loss: 3.1322 - val_accuracy: 0.5000
Epoch 4978/5000
 - 0s - loss: 0.1286 - accuracy: 0.9720 - val_loss: 3.1231 - val_accuracy: 0.3000
Epoch 4979/5000
 - 0

In [24]:
max(hist.history['val_accuracy'])

0.6000000238418579

In [25]:
from keras.models import load_model
b_model = load_model('best_2model.h5')
#b_model = load_model('./10testset_best_model/best_2model.h5')

answer = []
pred = []
answer_per = []
predict_per = []

for i in range(len(test2)) :
    test = test2[i].reshape(1,136)
    label = np.argmax(test_y[i])
    predict = b_model.predict(test)
    max_val = np.argmax(predict[0])
    answer.append(label)
    pred.append(max_val)
    answer_per.append(predict[0][label])
    predict_per.append(predict[0])
    print(label, max_val, predict[0][label], predict[0][max_val])

0 9 0.009944517 0.9774735
9 9 0.99286747 0.99286747
1 1 0.41369253 0.41369253
2 0 0.009631173 0.8671475
3 3 0.89544094 0.89544094
4 4 0.5895813 0.5895813
5 4 0.055505723 0.34839046
6 0 0.014376367 0.8052492
7 7 0.6303372 0.6303372
8 8 0.33257157 0.33257157


# 방법 3. 눈 주위 Landmark MLP

In [26]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint


# build the model
model = MLP_model()
mc = ModelCheckpoint('best_3model.h5', monitor='val_accuracy', mode='max', save_best_only=True)
# Fit the model
hist = model.fit(train34, train_y, validation_data=(test34, test_y), nb_epoch=5000, batch_size=200, verbose=2, callbacks=[mc])
# Final evaluation of the model
scores = model.evaluate(test34, test_y, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

c:\users\guswl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, activation="softmax", kernel_initializer="normal")`
  after removing the cwd from sys.path.
c:\users\guswl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # Remove the CWD from sys.path while we load stuff.


Train on 500 samples, validate on 10 samples
Epoch 1/5000
 - 0s - loss: 23.8941 - accuracy: 0.1060 - val_loss: 22.2346 - val_accuracy: 0.1000
Epoch 2/5000
 - 0s - loss: 18.6811 - accuracy: 0.0820 - val_loss: 16.7515 - val_accuracy: 0.1000
Epoch 3/5000
 - 0s - loss: 13.8937 - accuracy: 0.0960 - val_loss: 12.4536 - val_accuracy: 0.1000
Epoch 4/5000
 - 0s - loss: 10.2915 - accuracy: 0.1000 - val_loss: 10.4063 - val_accuracy: 0.1000
Epoch 5/5000
 - 0s - loss: 8.1201 - accuracy: 0.0960 - val_loss: 8.6527 - val_accuracy: 0.1000
Epoch 6/5000
 - 0s - loss: 6.0714 - accuracy: 0.1100 - val_loss: 6.2640 - val_accuracy: 0.1000
Epoch 7/5000
 - 0s - loss: 5.1382 - accuracy: 0.1140 - val_loss: 6.6689 - val_accuracy: 0.1000
Epoch 8/5000
 - 0s - loss: 5.2559 - accuracy: 0.0640 - val_loss: 5.5391 - val_accuracy: 0.1000
Epoch 9/5000
 - 0s - loss: 4.0063 - accuracy: 0.0660 - val_loss: 3.8748 - val_accuracy: 0.1000
Epoch 10/5000
 - 0s - loss: 3.3137 - accuracy: 0.1000 - val_loss: 4.1621 - val_accuracy: 0.1

Epoch 86/5000
 - 0s - loss: 2.0948 - accuracy: 0.2400 - val_loss: 2.2219 - val_accuracy: 0.2000
Epoch 87/5000
 - 0s - loss: 2.0919 - accuracy: 0.2460 - val_loss: 2.1828 - val_accuracy: 0.2000
Epoch 88/5000
 - 0s - loss: 2.0853 - accuracy: 0.2460 - val_loss: 2.2473 - val_accuracy: 0.2000
Epoch 89/5000
 - 0s - loss: 2.0893 - accuracy: 0.2640 - val_loss: 2.3110 - val_accuracy: 0.2000
Epoch 90/5000
 - 0s - loss: 2.0865 - accuracy: 0.2740 - val_loss: 2.1762 - val_accuracy: 0.2000
Epoch 91/5000
 - 0s - loss: 2.0769 - accuracy: 0.2720 - val_loss: 2.1848 - val_accuracy: 0.2000
Epoch 92/5000
 - 0s - loss: 2.0745 - accuracy: 0.2660 - val_loss: 2.2209 - val_accuracy: 0.2000
Epoch 93/5000
 - 0s - loss: 2.0615 - accuracy: 0.2780 - val_loss: 2.2511 - val_accuracy: 0.2000
Epoch 94/5000
 - 0s - loss: 2.0712 - accuracy: 0.2800 - val_loss: 2.2308 - val_accuracy: 0.2000
Epoch 95/5000
 - 0s - loss: 2.0619 - accuracy: 0.2940 - val_loss: 2.1995 - val_accuracy: 0.2000
Epoch 96/5000
 - 0s - loss: 2.0682 - acc

Epoch 171/5000
 - 0s - loss: 1.8408 - accuracy: 0.3940 - val_loss: 2.0344 - val_accuracy: 0.2000
Epoch 172/5000
 - 0s - loss: 1.8374 - accuracy: 0.4060 - val_loss: 2.0509 - val_accuracy: 0.2000
Epoch 173/5000
 - 0s - loss: 1.8318 - accuracy: 0.4020 - val_loss: 2.0292 - val_accuracy: 0.3000
Epoch 174/5000
 - 0s - loss: 1.8243 - accuracy: 0.4140 - val_loss: 2.0710 - val_accuracy: 0.3000
Epoch 175/5000
 - 0s - loss: 1.8431 - accuracy: 0.4020 - val_loss: 2.0361 - val_accuracy: 0.2000
Epoch 176/5000
 - 0s - loss: 1.8257 - accuracy: 0.4120 - val_loss: 2.0411 - val_accuracy: 0.2000
Epoch 177/5000
 - 0s - loss: 1.8279 - accuracy: 0.3960 - val_loss: 2.0401 - val_accuracy: 0.1000
Epoch 178/5000
 - 0s - loss: 1.8203 - accuracy: 0.4180 - val_loss: 2.0381 - val_accuracy: 0.3000
Epoch 179/5000
 - 0s - loss: 1.8210 - accuracy: 0.4160 - val_loss: 2.0550 - val_accuracy: 0.2000
Epoch 180/5000
 - 0s - loss: 1.8228 - accuracy: 0.4080 - val_loss: 2.0769 - val_accuracy: 0.2000
Epoch 181/5000
 - 0s - loss: 1

Epoch 256/5000
 - 0s - loss: 1.6628 - accuracy: 0.4800 - val_loss: 1.9293 - val_accuracy: 0.2000
Epoch 257/5000
 - 0s - loss: 1.6769 - accuracy: 0.4780 - val_loss: 1.9573 - val_accuracy: 0.3000
Epoch 258/5000
 - 0s - loss: 1.6639 - accuracy: 0.4880 - val_loss: 1.9207 - val_accuracy: 0.3000
Epoch 259/5000
 - 0s - loss: 1.6676 - accuracy: 0.4580 - val_loss: 1.8659 - val_accuracy: 0.3000
Epoch 260/5000
 - 0s - loss: 1.6661 - accuracy: 0.4840 - val_loss: 1.8946 - val_accuracy: 0.2000
Epoch 261/5000
 - 0s - loss: 1.6578 - accuracy: 0.4660 - val_loss: 1.9432 - val_accuracy: 0.4000
Epoch 262/5000
 - 0s - loss: 1.6720 - accuracy: 0.4860 - val_loss: 1.9211 - val_accuracy: 0.3000
Epoch 263/5000
 - 0s - loss: 1.6537 - accuracy: 0.4580 - val_loss: 1.8560 - val_accuracy: 0.3000
Epoch 264/5000
 - 0s - loss: 1.6666 - accuracy: 0.4500 - val_loss: 1.9656 - val_accuracy: 0.4000
Epoch 265/5000
 - 0s - loss: 1.6570 - accuracy: 0.4820 - val_loss: 1.9666 - val_accuracy: 0.3000
Epoch 266/5000
 - 0s - loss: 1

...

Epoch 4633/5000
 - 0s - loss: 0.5137 - accuracy: 0.8620 - val_loss: 1.2722 - val_accuracy: 0.7000
Epoch 4634/5000
 - 0s - loss: 0.5210 - accuracy: 0.8660 - val_loss: 1.3973 - val_accuracy: 0.4000
Epoch 4635/5000
 - 0s - loss: 0.5173 - accuracy: 0.8660 - val_loss: 1.1393 - val_accuracy: 0.6000
Epoch 4636/5000
 - 0s - loss: 0.5197 - accuracy: 0.8640 - val_loss: 1.1756 - val_accuracy: 0.5000
Epoch 4637/5000
 - 0s - loss: 0.5115 - accuracy: 0.8700 - val_loss: 1.3008 - val_accuracy: 0.7000
Epoch 4638/5000
 - 0s - loss: 0.5256 - accuracy: 0.8580 - val_loss: 1.2486 - val_accuracy: 0.6000
Epoch 4639/5000
 - 0s - loss: 0.5156 - accuracy: 0.8640 - val_loss: 1.3342 - val_accuracy: 0.6000
Epoch 4640/5000
 - 0s - loss: 0.5289 - accuracy: 0.8720 - val_loss: 1.2332 - val_accuracy: 0.7000
Epoch 4641/5000
 - 0s - loss: 0.5127 - accuracy: 0.8720 - val_loss: 1.1512 - val_accuracy: 0.5000
Epoch 4642/5000
 - 0s - loss: 0.5264 - accuracy: 0.8580 - val_loss: 1.2200 - val_accuracy: 0.5000
Epoch 4643/5000
 - 0

Epoch 4717/5000
 - 0s - loss: 0.5348 - accuracy: 0.8400 - val_loss: 1.2302 - val_accuracy: 0.6000
Epoch 4718/5000
 - 0s - loss: 0.5240 - accuracy: 0.8680 - val_loss: 1.1225 - val_accuracy: 0.6000
Epoch 4719/5000
 - 0s - loss: 0.5162 - accuracy: 0.8740 - val_loss: 1.2022 - val_accuracy: 0.5000
Epoch 4720/5000
 - 0s - loss: 0.5322 - accuracy: 0.8540 - val_loss: 1.2331 - val_accuracy: 0.6000
Epoch 4721/5000
 - 0s - loss: 0.5104 - accuracy: 0.8640 - val_loss: 1.2884 - val_accuracy: 0.5000
Epoch 4722/5000
 - 0s - loss: 0.5118 - accuracy: 0.8680 - val_loss: 1.2176 - val_accuracy: 0.6000
Epoch 4723/5000
 - 0s - loss: 0.5158 - accuracy: 0.8560 - val_loss: 1.1410 - val_accuracy: 0.5000
Epoch 4724/5000
 - 0s - loss: 0.5098 - accuracy: 0.8700 - val_loss: 1.2235 - val_accuracy: 0.6000
Epoch 4725/5000
 - 0s - loss: 0.5133 - accuracy: 0.8660 - val_loss: 1.3413 - val_accuracy: 0.5000
Epoch 4726/5000
 - 0s - loss: 0.5136 - accuracy: 0.8720 - val_loss: 1.2300 - val_accuracy: 0.6000
Epoch 4727/5000
 - 0

Epoch 4801/5000
 - 0s - loss: 0.5062 - accuracy: 0.8700 - val_loss: 1.4943 - val_accuracy: 0.4000
Epoch 4802/5000
 - 0s - loss: 0.5232 - accuracy: 0.8440 - val_loss: 1.1913 - val_accuracy: 0.6000
Epoch 4803/5000
 - 0s - loss: 0.5115 - accuracy: 0.8640 - val_loss: 1.0816 - val_accuracy: 0.7000
Epoch 4804/5000
 - 0s - loss: 0.5035 - accuracy: 0.8640 - val_loss: 1.3419 - val_accuracy: 0.6000
Epoch 4805/5000
 - 0s - loss: 0.5124 - accuracy: 0.8520 - val_loss: 1.2312 - val_accuracy: 0.6000
Epoch 4806/5000
 - 0s - loss: 0.5065 - accuracy: 0.8680 - val_loss: 1.1809 - val_accuracy: 0.6000
Epoch 4807/5000
 - 0s - loss: 0.5100 - accuracy: 0.8720 - val_loss: 1.2877 - val_accuracy: 0.6000
Epoch 4808/5000
 - 0s - loss: 0.5048 - accuracy: 0.8740 - val_loss: 1.2902 - val_accuracy: 0.5000
Epoch 4809/5000
 - 0s - loss: 0.5045 - accuracy: 0.8740 - val_loss: 1.2419 - val_accuracy: 0.7000
Epoch 4810/5000
 - 0s - loss: 0.5033 - accuracy: 0.8660 - val_loss: 1.3095 - val_accuracy: 0.5000
Epoch 4811/5000
 - 0

Epoch 4885/5000
 - 0s - loss: 0.5062 - accuracy: 0.8680 - val_loss: 1.3061 - val_accuracy: 0.5000
Epoch 4886/5000
 - 0s - loss: 0.5099 - accuracy: 0.8620 - val_loss: 1.2001 - val_accuracy: 0.6000
Epoch 4887/5000
 - 0s - loss: 0.5149 - accuracy: 0.8520 - val_loss: 1.1575 - val_accuracy: 0.7000
Epoch 4888/5000
 - 0s - loss: 0.5018 - accuracy: 0.8600 - val_loss: 1.3965 - val_accuracy: 0.5000
Epoch 4889/5000
 - 0s - loss: 0.5149 - accuracy: 0.8560 - val_loss: 1.0999 - val_accuracy: 0.7000
Epoch 4890/5000
 - 0s - loss: 0.5088 - accuracy: 0.8680 - val_loss: 1.0809 - val_accuracy: 0.6000
Epoch 4891/5000
 - 0s - loss: 0.5170 - accuracy: 0.8620 - val_loss: 1.3615 - val_accuracy: 0.5000
Epoch 4892/5000
 - 0s - loss: 0.5068 - accuracy: 0.8680 - val_loss: 1.2791 - val_accuracy: 0.5000
Epoch 4893/5000
 - 0s - loss: 0.5117 - accuracy: 0.8560 - val_loss: 1.1882 - val_accuracy: 0.6000
Epoch 4894/5000
 - 0s - loss: 0.5046 - accuracy: 0.8700 - val_loss: 1.1562 - val_accuracy: 0.6000
Epoch 4895/5000
 - 0

Epoch 4969/5000
 - 0s - loss: 0.5064 - accuracy: 0.8680 - val_loss: 1.2303 - val_accuracy: 0.6000
Epoch 4970/5000
 - 0s - loss: 0.5018 - accuracy: 0.8780 - val_loss: 1.2787 - val_accuracy: 0.6000
Epoch 4971/5000
 - 0s - loss: 0.5023 - accuracy: 0.8640 - val_loss: 1.2125 - val_accuracy: 0.5000
Epoch 4972/5000
 - 0s - loss: 0.5012 - accuracy: 0.8700 - val_loss: 1.1913 - val_accuracy: 0.5000
Epoch 4973/5000
 - 0s - loss: 0.5051 - accuracy: 0.8780 - val_loss: 1.2171 - val_accuracy: 0.6000
Epoch 4974/5000
 - 0s - loss: 0.5073 - accuracy: 0.8580 - val_loss: 1.2447 - val_accuracy: 0.6000
Epoch 4975/5000
 - 0s - loss: 0.5094 - accuracy: 0.8620 - val_loss: 1.2707 - val_accuracy: 0.6000
Epoch 4976/5000
 - 0s - loss: 0.5202 - accuracy: 0.8520 - val_loss: 1.2134 - val_accuracy: 0.7000
Epoch 4977/5000
 - 0s - loss: 0.5104 - accuracy: 0.8620 - val_loss: 1.2397 - val_accuracy: 0.5000
Epoch 4978/5000
 - 0s - loss: 0.5158 - accuracy: 0.8660 - val_loss: 1.2463 - val_accuracy: 0.5000
Epoch 4979/5000
 - 0

In [27]:
max(hist.history['val_accuracy'])

0.800000011920929

In [28]:
from keras.models import load_model
b_model = load_model('best_3model.h5')
#b_model = load_model('./10testset_best_model/best_3model.h5')

answer = []
pred = []
answer_per = []
predict_per = []

for i in range(len(test34)) :
    test = test34[i].reshape(1,50)
    label = np.argmax(test_y[i])
    predict = b_model.predict(test)
    max_val = np.argmax(predict[0])
    answer.append(label)
    pred.append(max_val)
    answer_per.append(predict[0][label])
    predict_per.append(predict[0])
    print(label, max_val, predict[0][label], predict[0][max_val])

0 3 0.008534824 0.857162
9 9 0.9462566 0.9462566
1 7 0.0534488 0.57705057
2 2 0.23069012 0.23069012
3 3 0.9971685 0.9971685
4 4 0.55231667 0.55231667
5 5 0.57926387 0.57926387
6 6 0.62546873 0.62546873
7 7 0.46537536 0.46537536
8 8 0.58321565 0.58321565


# 방법 4. 눈 주위 Landmark DNN

In [29]:
def good_model():
    # create model
    model = Sequential()
    model.add(Dense(400, input_dim=50, init='normal', activation='relu'))
    model.add(Dense(100, init='normal', activation='relu'))
    model.add(Dense(10, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [30]:
# build the model
model2 = good_model()
mc = ModelCheckpoint('best_4model.h5', monitor='val_accuracy', mode='max', save_best_only=True)
# Fit the model
hist = model2.fit(train34, train_y, validation_data=(test34, test_y), nb_epoch=5000, batch_size=200, verbose=2, callbacks=[mc])
# Final evaluation of the model
scores = model2.evaluate(test34, test_y, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

c:\users\guswl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(400, input_dim=50, activation="relu", kernel_initializer="normal")`
  after removing the cwd from sys.path.
c:\users\guswl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, activation="relu", kernel_initializer="normal")`
  """
c:\users\guswl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, activation="softmax", kernel_initializer="normal")`
  
c:\users\guswl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Train on 500 samples, validate on 10 samples
Epoch 1/5000
 - 0s - loss: 4.6853 - accuracy: 0.1000 - val_loss: 3.7386 - val_accuracy: 0.2000
Epoch 2/5000
 - 0s - loss: 3.6114 - accuracy: 0.1080 - val_loss: 3.3296 - val_accuracy: 0.1000
Epoch 3/5000
 - 0s - loss: 3.1159 - accuracy: 0.1180 - val_loss: 2.6597 - val_accuracy: 0.2000
Epoch 4/5000
 - 0s - loss: 2.5603 - accuracy: 0.1260 - val_loss: 2.4489 - val_accuracy: 0.1000
Epoch 5/5000
 - 0s - loss: 2.5874 - accuracy: 0.1080 - val_loss: 2.5850 - val_accuracy: 0.1000
Epoch 6/5000
 - 0s - loss: 2.5397 - accuracy: 0.1160 - val_loss: 2.2852 - val_accuracy: 0.2000
Epoch 7/5000
 - 0s - loss: 2.3406 - accuracy: 0.1480 - val_loss: 2.2943 - val_accuracy: 0.1000
Epoch 8/5000
 - 0s - loss: 2.3269 - accuracy: 0.1260 - val_loss: 2.3821 - val_accuracy: 0.1000
Epoch 9/5000
 - 0s - loss: 2.3305 - accuracy: 0.1420 - val_loss: 2.2659 - val_accuracy: 0.1000
Epoch 10/5000
 - 0s - loss: 2.2654 - accuracy: 0.1520 - val_loss: 2.1979 - val_accuracy: 0.1000
Epoc

Epoch 86/5000
 - 0s - loss: 1.7243 - accuracy: 0.3800 - val_loss: 2.0323 - val_accuracy: 0.2000
Epoch 87/5000
 - 0s - loss: 1.7314 - accuracy: 0.3900 - val_loss: 1.8730 - val_accuracy: 0.3000
Epoch 88/5000
 - 0s - loss: 1.7432 - accuracy: 0.3600 - val_loss: 1.7630 - val_accuracy: 0.2000
Epoch 89/5000
 - 0s - loss: 1.7202 - accuracy: 0.4020 - val_loss: 1.9844 - val_accuracy: 0.2000
Epoch 90/5000
 - 0s - loss: 1.7335 - accuracy: 0.3860 - val_loss: 2.2177 - val_accuracy: 0.2000
Epoch 91/5000
 - 0s - loss: 1.7656 - accuracy: 0.4040 - val_loss: 1.6979 - val_accuracy: 0.4000
Epoch 92/5000
 - 0s - loss: 1.8115 - accuracy: 0.3580 - val_loss: 1.9401 - val_accuracy: 0.4000
Epoch 93/5000
 - 0s - loss: 1.7251 - accuracy: 0.4160 - val_loss: 1.9013 - val_accuracy: 0.2000
Epoch 94/5000
 - 0s - loss: 1.6886 - accuracy: 0.3760 - val_loss: 1.7179 - val_accuracy: 0.3000
Epoch 95/5000
 - 0s - loss: 1.6783 - accuracy: 0.4300 - val_loss: 2.1278 - val_accuracy: 0.1000
Epoch 96/5000
 - 0s - loss: 1.6706 - acc

Epoch 171/5000
 - 0s - loss: 1.2127 - accuracy: 0.5740 - val_loss: 1.8129 - val_accuracy: 0.3000
Epoch 172/5000
 - 0s - loss: 1.1924 - accuracy: 0.6040 - val_loss: 1.6683 - val_accuracy: 0.4000
Epoch 173/5000
 - 0s - loss: 1.1759 - accuracy: 0.6300 - val_loss: 1.8480 - val_accuracy: 0.2000
Epoch 174/5000
 - 0s - loss: 1.1678 - accuracy: 0.6260 - val_loss: 1.6863 - val_accuracy: 0.4000
Epoch 175/5000
 - 0s - loss: 1.1668 - accuracy: 0.6300 - val_loss: 1.6295 - val_accuracy: 0.4000
Epoch 176/5000
 - 0s - loss: 1.1830 - accuracy: 0.6160 - val_loss: 1.7738 - val_accuracy: 0.4000
Epoch 177/5000
 - 0s - loss: 1.1935 - accuracy: 0.5860 - val_loss: 1.6998 - val_accuracy: 0.3000
Epoch 178/5000
 - 0s - loss: 1.1535 - accuracy: 0.6340 - val_loss: 1.8869 - val_accuracy: 0.3000
Epoch 179/5000
 - 0s - loss: 1.1465 - accuracy: 0.6300 - val_loss: 1.5587 - val_accuracy: 0.5000
Epoch 180/5000
 - 0s - loss: 1.1590 - accuracy: 0.6240 - val_loss: 1.9115 - val_accuracy: 0.3000
Epoch 181/5000
 - 0s - loss: 1

Epoch 256/5000
 - 0s - loss: 0.8540 - accuracy: 0.7380 - val_loss: 1.2109 - val_accuracy: 0.6000
Epoch 257/5000
 - 0s - loss: 0.8913 - accuracy: 0.7020 - val_loss: 1.4817 - val_accuracy: 0.3000
Epoch 258/5000
 - 0s - loss: 0.8490 - accuracy: 0.7400 - val_loss: 1.4230 - val_accuracy: 0.5000
Epoch 259/5000
 - 0s - loss: 0.8403 - accuracy: 0.7520 - val_loss: 1.4521 - val_accuracy: 0.4000
Epoch 260/5000
 - 0s - loss: 0.8508 - accuracy: 0.7300 - val_loss: 1.3009 - val_accuracy: 0.6000
Epoch 261/5000
 - 0s - loss: 0.8525 - accuracy: 0.7180 - val_loss: 1.5496 - val_accuracy: 0.3000
Epoch 262/5000
 - 0s - loss: 0.8633 - accuracy: 0.7500 - val_loss: 1.5109 - val_accuracy: 0.3000
Epoch 263/5000
 - 0s - loss: 0.8374 - accuracy: 0.7320 - val_loss: 1.2842 - val_accuracy: 0.5000
Epoch 264/5000
 - 0s - loss: 0.8848 - accuracy: 0.6880 - val_loss: 1.5554 - val_accuracy: 0.4000
Epoch 265/5000
 - 0s - loss: 0.8709 - accuracy: 0.7040 - val_loss: 1.3085 - val_accuracy: 0.5000
Epoch 266/5000
 - 0s - loss: 0

...

Epoch 4633/5000
 - 0s - loss: 0.0388 - accuracy: 1.0000 - val_loss: 1.4748 - val_accuracy: 0.6000
Epoch 4634/5000
 - 0s - loss: 0.0382 - accuracy: 1.0000 - val_loss: 1.5068 - val_accuracy: 0.4000
Epoch 4635/5000
 - 0s - loss: 0.0421 - accuracy: 0.9980 - val_loss: 1.3899 - val_accuracy: 0.6000
Epoch 4636/5000
 - 0s - loss: 0.0391 - accuracy: 1.0000 - val_loss: 1.4545 - val_accuracy: 0.4000
Epoch 4637/5000
 - 0s - loss: 0.0390 - accuracy: 1.0000 - val_loss: 1.6007 - val_accuracy: 0.5000
Epoch 4638/5000
 - 0s - loss: 0.0394 - accuracy: 0.9980 - val_loss: 1.3950 - val_accuracy: 0.4000
Epoch 4639/5000
 - 0s - loss: 0.0404 - accuracy: 0.9980 - val_loss: 1.5486 - val_accuracy: 0.6000
Epoch 4640/5000
 - 0s - loss: 0.0411 - accuracy: 0.9980 - val_loss: 1.3627 - val_accuracy: 0.6000
Epoch 4641/5000
 - 0s - loss: 0.0386 - accuracy: 0.9980 - val_loss: 1.4412 - val_accuracy: 0.4000
Epoch 4642/5000
 - 0s - loss: 0.0396 - accuracy: 1.0000 - val_loss: 1.5362 - val_accuracy: 0.5000
Epoch 4643/5000
 - 0

Epoch 4717/5000
 - 0s - loss: 0.0346 - accuracy: 0.9980 - val_loss: 1.6791 - val_accuracy: 0.5000
Epoch 4718/5000
 - 0s - loss: 0.0470 - accuracy: 0.9920 - val_loss: 1.8827 - val_accuracy: 0.5000
Epoch 4719/5000
 - 0s - loss: 0.0575 - accuracy: 0.9900 - val_loss: 1.5502 - val_accuracy: 0.4000
Epoch 4720/5000
 - 0s - loss: 0.0708 - accuracy: 0.9840 - val_loss: 1.7611 - val_accuracy: 0.7000
Epoch 4721/5000
 - 0s - loss: 0.0839 - accuracy: 0.9880 - val_loss: 1.7028 - val_accuracy: 0.4000
Epoch 4722/5000
 - 0s - loss: 0.0843 - accuracy: 0.9780 - val_loss: 2.0175 - val_accuracy: 0.6000
Epoch 4723/5000
 - 0s - loss: 0.0687 - accuracy: 0.9880 - val_loss: 1.0029 - val_accuracy: 0.5000
Epoch 4724/5000
 - 0s - loss: 0.0820 - accuracy: 0.9840 - val_loss: 2.3993 - val_accuracy: 0.6000
Epoch 4725/5000
 - 0s - loss: 0.1102 - accuracy: 0.9600 - val_loss: 0.9165 - val_accuracy: 0.6000
Epoch 4726/5000
 - 0s - loss: 0.1387 - accuracy: 0.9440 - val_loss: 1.5122 - val_accuracy: 0.6000
Epoch 4727/5000
 - 0

Epoch 4801/5000
 - 0s - loss: 0.0245 - accuracy: 1.0000 - val_loss: 1.6076 - val_accuracy: 0.5000
Epoch 4802/5000
 - 0s - loss: 0.0241 - accuracy: 1.0000 - val_loss: 1.6163 - val_accuracy: 0.6000
Epoch 4803/5000
 - 0s - loss: 0.0238 - accuracy: 1.0000 - val_loss: 1.4992 - val_accuracy: 0.6000
Epoch 4804/5000
 - 0s - loss: 0.0236 - accuracy: 1.0000 - val_loss: 1.5225 - val_accuracy: 0.6000
Epoch 4805/5000
 - 0s - loss: 0.0237 - accuracy: 1.0000 - val_loss: 1.7220 - val_accuracy: 0.5000
Epoch 4806/5000
 - 0s - loss: 0.0233 - accuracy: 1.0000 - val_loss: 1.6256 - val_accuracy: 0.5000
Epoch 4807/5000
 - 0s - loss: 0.0232 - accuracy: 1.0000 - val_loss: 1.6389 - val_accuracy: 0.6000
Epoch 4808/5000
 - 0s - loss: 0.0230 - accuracy: 1.0000 - val_loss: 1.6176 - val_accuracy: 0.6000
Epoch 4809/5000
 - 0s - loss: 0.0236 - accuracy: 1.0000 - val_loss: 1.6180 - val_accuracy: 0.5000
Epoch 4810/5000
 - 0s - loss: 0.0242 - accuracy: 0.9980 - val_loss: 1.6126 - val_accuracy: 0.5000
Epoch 4811/5000
 - 0

Epoch 4885/5000
 - 0s - loss: 0.0215 - accuracy: 1.0000 - val_loss: 1.5625 - val_accuracy: 0.5000
Epoch 4886/5000
 - 0s - loss: 0.0206 - accuracy: 1.0000 - val_loss: 1.7575 - val_accuracy: 0.5000
Epoch 4887/5000
 - 0s - loss: 0.0205 - accuracy: 1.0000 - val_loss: 1.5903 - val_accuracy: 0.5000
Epoch 4888/5000
 - 0s - loss: 0.0195 - accuracy: 1.0000 - val_loss: 1.5001 - val_accuracy: 0.6000
Epoch 4889/5000
 - 0s - loss: 0.0194 - accuracy: 1.0000 - val_loss: 1.4747 - val_accuracy: 0.6000
Epoch 4890/5000
 - 0s - loss: 0.0191 - accuracy: 1.0000 - val_loss: 1.5578 - val_accuracy: 0.5000
Epoch 4891/5000
 - 0s - loss: 0.0191 - accuracy: 1.0000 - val_loss: 1.5651 - val_accuracy: 0.5000
Epoch 4892/5000
 - 0s - loss: 0.0189 - accuracy: 1.0000 - val_loss: 1.6247 - val_accuracy: 0.5000
Epoch 4893/5000
 - 0s - loss: 0.0184 - accuracy: 1.0000 - val_loss: 1.6583 - val_accuracy: 0.5000
Epoch 4894/5000
 - 0s - loss: 0.0187 - accuracy: 1.0000 - val_loss: 1.7126 - val_accuracy: 0.5000
Epoch 4895/5000
 - 0

Epoch 4969/5000
 - 0s - loss: 0.0159 - accuracy: 1.0000 - val_loss: 1.5116 - val_accuracy: 0.5000
Epoch 4970/5000
 - 0s - loss: 0.0163 - accuracy: 1.0000 - val_loss: 1.6727 - val_accuracy: 0.5000
Epoch 4971/5000
 - 0s - loss: 0.0159 - accuracy: 1.0000 - val_loss: 1.7416 - val_accuracy: 0.5000
Epoch 4972/5000
 - 0s - loss: 0.0167 - accuracy: 1.0000 - val_loss: 1.5850 - val_accuracy: 0.5000
Epoch 4973/5000
 - 0s - loss: 0.0162 - accuracy: 1.0000 - val_loss: 1.6391 - val_accuracy: 0.5000
Epoch 4974/5000
 - 0s - loss: 0.0158 - accuracy: 1.0000 - val_loss: 1.6043 - val_accuracy: 0.5000
Epoch 4975/5000
 - 0s - loss: 0.0167 - accuracy: 1.0000 - val_loss: 1.5004 - val_accuracy: 0.6000
Epoch 4976/5000
 - 0s - loss: 0.0165 - accuracy: 1.0000 - val_loss: 1.7509 - val_accuracy: 0.5000
Epoch 4977/5000
 - 0s - loss: 0.0164 - accuracy: 1.0000 - val_loss: 1.6512 - val_accuracy: 0.5000
Epoch 4978/5000
 - 0s - loss: 0.0173 - accuracy: 1.0000 - val_loss: 1.5481 - val_accuracy: 0.5000
Epoch 4979/5000
 - 0

In [31]:
max(hist.history['val_accuracy'])

0.8999999761581421

In [32]:
from keras.models import load_model

b_model = load_model('best_4model.h5')
#b_model = load_model('./10testset_best_model/best_4model.h5')

answer = []
pred = []
answer_per = []
predict_per = []

for i in range(len(test34)) :
    test = test34[i].reshape(1,50)
    label = np.argmax(test_y[i])
    predict = b_model.predict(test)
    max_val = np.argmax(predict[0])
    answer.append(label)
    pred.append(max_val)
    answer_per.append(predict[0][label])
    predict_per.append(predict[0])
    print(label, max_val, predict[0][label], predict[0][max_val])

0 0 0.62893623 0.62893623
9 9 0.9995535 0.9995535
1 1 0.9592337 0.9592337
2 2 0.3517863 0.3517863
3 3 0.99999976 0.99999976
4 4 0.7689948 0.7689948
5 5 0.639299 0.639299
6 6 0.99588495 0.99588495
7 3 0.00041396637 0.79948777
8 8 0.98996896 0.98996896


# 방법 5. 128 embedding MLP

In [33]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint


# build the model
model = MLP_model()
mc = ModelCheckpoint('best_5model.h5', monitor='val_accuracy', mode='max', save_best_only=True)
# Fit the model
hist = model.fit(train56, train_y, validation_data=(test56, test_y), nb_epoch=5000, batch_size=200, verbose=2,
                  callbacks=[mc])
# Final evaluation of the model
scores = model.evaluate(test56, test_y, verbose=1)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

c:\users\guswl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, activation="softmax", kernel_initializer="normal")`
  after removing the cwd from sys.path.
c:\users\guswl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # This is added back by InteractiveShellApp.init_path()


Train on 500 samples, validate on 10 samples
Epoch 1/5000
 - 0s - loss: 2.3004 - accuracy: 0.1500 - val_loss: 2.3006 - val_accuracy: 0.1000
Epoch 2/5000
 - 0s - loss: 2.2927 - accuracy: 0.1840 - val_loss: 2.2963 - val_accuracy: 0.1000
Epoch 3/5000
 - 0s - loss: 2.2853 - accuracy: 0.2140 - val_loss: 2.2919 - val_accuracy: 0.1000
Epoch 4/5000
 - 0s - loss: 2.2779 - accuracy: 0.2560 - val_loss: 2.2877 - val_accuracy: 0.1000
Epoch 5/5000
 - 0s - loss: 2.2707 - accuracy: 0.3100 - val_loss: 2.2835 - val_accuracy: 0.1000
Epoch 6/5000
 - 0s - loss: 2.2634 - accuracy: 0.3440 - val_loss: 2.2794 - val_accuracy: 0.1000
Epoch 7/5000
 - 0s - loss: 2.2563 - accuracy: 0.3820 - val_loss: 2.2753 - val_accuracy: 0.1000
Epoch 8/5000
 - 0s - loss: 2.2491 - accuracy: 0.4260 - val_loss: 2.2711 - val_accuracy: 0.1000
Epoch 9/5000
 - 0s - loss: 2.2419 - accuracy: 0.4360 - val_loss: 2.2670 - val_accuracy: 0.1000
Epoch 10/5000
 - 0s - loss: 2.2349 - accuracy: 0.4580 - val_loss: 2.2629 - val_accuracy: 0.1000
Epoc

Epoch 86/5000
 - 0s - loss: 1.8024 - accuracy: 0.7460 - val_loss: 2.0158 - val_accuracy: 0.5000
Epoch 87/5000
 - 0s - loss: 1.7979 - accuracy: 0.7480 - val_loss: 2.0134 - val_accuracy: 0.4000
Epoch 88/5000
 - 0s - loss: 1.7934 - accuracy: 0.7480 - val_loss: 2.0109 - val_accuracy: 0.4000
Epoch 89/5000
 - 0s - loss: 1.7889 - accuracy: 0.7480 - val_loss: 2.0086 - val_accuracy: 0.4000
Epoch 90/5000
 - 0s - loss: 1.7845 - accuracy: 0.7480 - val_loss: 2.0063 - val_accuracy: 0.4000
Epoch 91/5000
 - 0s - loss: 1.7800 - accuracy: 0.7500 - val_loss: 2.0039 - val_accuracy: 0.4000
Epoch 92/5000
 - 0s - loss: 1.7756 - accuracy: 0.7500 - val_loss: 2.0014 - val_accuracy: 0.4000
Epoch 93/5000
 - 0s - loss: 1.7712 - accuracy: 0.7520 - val_loss: 1.9991 - val_accuracy: 0.4000
Epoch 94/5000
 - 0s - loss: 1.7669 - accuracy: 0.7580 - val_loss: 1.9969 - val_accuracy: 0.4000
Epoch 95/5000
 - 0s - loss: 1.7625 - accuracy: 0.7600 - val_loss: 1.9947 - val_accuracy: 0.4000
Epoch 96/5000
 - 0s - loss: 1.7582 - acc

Epoch 171/5000
 - 0s - loss: 1.4884 - accuracy: 0.7880 - val_loss: 1.8572 - val_accuracy: 0.4000
Epoch 172/5000
 - 0s - loss: 1.4853 - accuracy: 0.7860 - val_loss: 1.8560 - val_accuracy: 0.4000
Epoch 173/5000
 - 0s - loss: 1.4823 - accuracy: 0.7840 - val_loss: 1.8545 - val_accuracy: 0.4000
Epoch 174/5000
 - 0s - loss: 1.4794 - accuracy: 0.7860 - val_loss: 1.8532 - val_accuracy: 0.4000
Epoch 175/5000
 - 0s - loss: 1.4764 - accuracy: 0.7860 - val_loss: 1.8520 - val_accuracy: 0.4000
Epoch 176/5000
 - 0s - loss: 1.4735 - accuracy: 0.7880 - val_loss: 1.8506 - val_accuracy: 0.4000
Epoch 177/5000
 - 0s - loss: 1.4706 - accuracy: 0.7880 - val_loss: 1.8494 - val_accuracy: 0.4000
Epoch 178/5000
 - 0s - loss: 1.4676 - accuracy: 0.7900 - val_loss: 1.8482 - val_accuracy: 0.4000
Epoch 179/5000
 - 0s - loss: 1.4647 - accuracy: 0.7880 - val_loss: 1.8470 - val_accuracy: 0.4000
Epoch 180/5000
 - 0s - loss: 1.4619 - accuracy: 0.7880 - val_loss: 1.8459 - val_accuracy: 0.4000
Epoch 181/5000
 - 0s - loss: 1

Epoch 256/5000
 - 0s - loss: 1.2745 - accuracy: 0.7980 - val_loss: 1.7758 - val_accuracy: 0.4000
Epoch 257/5000
 - 0s - loss: 1.2724 - accuracy: 0.7980 - val_loss: 1.7753 - val_accuracy: 0.4000
Epoch 258/5000
 - 0s - loss: 1.2704 - accuracy: 0.8000 - val_loss: 1.7749 - val_accuracy: 0.4000
Epoch 259/5000
 - 0s - loss: 1.2682 - accuracy: 0.8000 - val_loss: 1.7745 - val_accuracy: 0.4000
Epoch 260/5000
 - 0s - loss: 1.2661 - accuracy: 0.8000 - val_loss: 1.7744 - val_accuracy: 0.4000
Epoch 261/5000
 - 0s - loss: 1.2641 - accuracy: 0.8000 - val_loss: 1.7742 - val_accuracy: 0.4000
Epoch 262/5000
 - 0s - loss: 1.2620 - accuracy: 0.7980 - val_loss: 1.7740 - val_accuracy: 0.4000
Epoch 263/5000
 - 0s - loss: 1.2600 - accuracy: 0.7980 - val_loss: 1.7736 - val_accuracy: 0.4000
Epoch 264/5000
 - 0s - loss: 1.2579 - accuracy: 0.8000 - val_loss: 1.7731 - val_accuracy: 0.4000
Epoch 265/5000
 - 0s - loss: 1.2559 - accuracy: 0.8000 - val_loss: 1.7727 - val_accuracy: 0.4000
Epoch 266/5000
 - 0s - loss: 1

...

Epoch 4633/5000
 - 0s - loss: 0.2786 - accuracy: 0.9260 - val_loss: 3.4780 - val_accuracy: 0.4000
Epoch 4634/5000
 - 0s - loss: 0.2785 - accuracy: 0.9260 - val_loss: 3.4788 - val_accuracy: 0.4000
Epoch 4635/5000
 - 0s - loss: 0.2785 - accuracy: 0.9260 - val_loss: 3.4790 - val_accuracy: 0.4000
Epoch 4636/5000
 - 0s - loss: 0.2785 - accuracy: 0.9260 - val_loss: 3.4779 - val_accuracy: 0.4000
Epoch 4637/5000
 - 0s - loss: 0.2785 - accuracy: 0.9260 - val_loss: 3.4785 - val_accuracy: 0.4000
Epoch 4638/5000
 - 0s - loss: 0.2784 - accuracy: 0.9260 - val_loss: 3.4796 - val_accuracy: 0.4000
Epoch 4639/5000
 - 0s - loss: 0.2784 - accuracy: 0.9260 - val_loss: 3.4805 - val_accuracy: 0.4000
Epoch 4640/5000
 - 0s - loss: 0.2784 - accuracy: 0.9260 - val_loss: 3.4815 - val_accuracy: 0.4000
Epoch 4641/5000
 - 0s - loss: 0.2784 - accuracy: 0.9260 - val_loss: 3.4817 - val_accuracy: 0.4000
Epoch 4642/5000
 - 0s - loss: 0.2783 - accuracy: 0.9260 - val_loss: 3.4822 - val_accuracy: 0.4000
Epoch 4643/5000
 - 0

Epoch 4717/5000
 - 0s - loss: 0.2765 - accuracy: 0.9280 - val_loss: 3.5016 - val_accuracy: 0.4000
Epoch 4718/5000
 - 0s - loss: 0.2765 - accuracy: 0.9280 - val_loss: 3.5023 - val_accuracy: 0.4000
Epoch 4719/5000
 - 0s - loss: 0.2764 - accuracy: 0.9280 - val_loss: 3.5036 - val_accuracy: 0.4000
Epoch 4720/5000
 - 0s - loss: 0.2764 - accuracy: 0.9280 - val_loss: 3.5049 - val_accuracy: 0.4000
Epoch 4721/5000
 - 0s - loss: 0.2764 - accuracy: 0.9280 - val_loss: 3.5064 - val_accuracy: 0.4000
Epoch 4722/5000
 - 0s - loss: 0.2764 - accuracy: 0.9260 - val_loss: 3.5084 - val_accuracy: 0.4000
Epoch 4723/5000
 - 0s - loss: 0.2763 - accuracy: 0.9260 - val_loss: 3.5098 - val_accuracy: 0.4000
Epoch 4724/5000
 - 0s - loss: 0.2763 - accuracy: 0.9260 - val_loss: 3.5104 - val_accuracy: 0.4000
Epoch 4725/5000
 - 0s - loss: 0.2763 - accuracy: 0.9260 - val_loss: 3.5108 - val_accuracy: 0.4000
Epoch 4726/5000
 - 0s - loss: 0.2763 - accuracy: 0.9260 - val_loss: 3.5112 - val_accuracy: 0.4000
Epoch 4727/5000
 - 0

Epoch 4801/5000
 - 0s - loss: 0.2745 - accuracy: 0.9300 - val_loss: 3.5263 - val_accuracy: 0.4000
Epoch 4802/5000
 - 0s - loss: 0.2744 - accuracy: 0.9300 - val_loss: 3.5264 - val_accuracy: 0.4000
Epoch 4803/5000
 - 0s - loss: 0.2744 - accuracy: 0.9300 - val_loss: 3.5268 - val_accuracy: 0.4000
Epoch 4804/5000
 - 0s - loss: 0.2744 - accuracy: 0.9300 - val_loss: 3.5277 - val_accuracy: 0.4000
Epoch 4805/5000
 - 0s - loss: 0.2744 - accuracy: 0.9300 - val_loss: 3.5282 - val_accuracy: 0.4000
Epoch 4806/5000
 - 0s - loss: 0.2744 - accuracy: 0.9300 - val_loss: 3.5280 - val_accuracy: 0.4000
Epoch 4807/5000
 - 0s - loss: 0.2743 - accuracy: 0.9300 - val_loss: 3.5285 - val_accuracy: 0.4000
Epoch 4808/5000
 - 0s - loss: 0.2743 - accuracy: 0.9300 - val_loss: 3.5291 - val_accuracy: 0.4000
Epoch 4809/5000
 - 0s - loss: 0.2743 - accuracy: 0.9300 - val_loss: 3.5299 - val_accuracy: 0.4000
Epoch 4810/5000
 - 0s - loss: 0.2743 - accuracy: 0.9300 - val_loss: 3.5298 - val_accuracy: 0.4000
Epoch 4811/5000
 - 0

Epoch 4885/5000
 - 0s - loss: 0.2725 - accuracy: 0.9280 - val_loss: 3.5532 - val_accuracy: 0.4000
Epoch 4886/5000
 - 0s - loss: 0.2725 - accuracy: 0.9280 - val_loss: 3.5552 - val_accuracy: 0.4000
Epoch 4887/5000
 - 0s - loss: 0.2725 - accuracy: 0.9300 - val_loss: 3.5560 - val_accuracy: 0.4000
Epoch 4888/5000
 - 0s - loss: 0.2725 - accuracy: 0.9300 - val_loss: 3.5562 - val_accuracy: 0.4000
Epoch 4889/5000
 - 0s - loss: 0.2724 - accuracy: 0.9300 - val_loss: 3.5572 - val_accuracy: 0.4000
Epoch 4890/5000
 - 0s - loss: 0.2724 - accuracy: 0.9300 - val_loss: 3.5577 - val_accuracy: 0.4000
Epoch 4891/5000
 - 0s - loss: 0.2724 - accuracy: 0.9300 - val_loss: 3.5580 - val_accuracy: 0.4000
Epoch 4892/5000
 - 0s - loss: 0.2724 - accuracy: 0.9300 - val_loss: 3.5590 - val_accuracy: 0.4000
Epoch 4893/5000
 - 0s - loss: 0.2723 - accuracy: 0.9300 - val_loss: 3.5591 - val_accuracy: 0.4000
Epoch 4894/5000
 - 0s - loss: 0.2723 - accuracy: 0.9300 - val_loss: 3.5598 - val_accuracy: 0.4000
Epoch 4895/5000
 - 0

Epoch 4969/5000
 - 0s - loss: 0.2706 - accuracy: 0.9300 - val_loss: 3.5760 - val_accuracy: 0.4000
Epoch 4970/5000
 - 0s - loss: 0.2706 - accuracy: 0.9300 - val_loss: 3.5775 - val_accuracy: 0.4000
Epoch 4971/5000
 - 0s - loss: 0.2705 - accuracy: 0.9300 - val_loss: 3.5783 - val_accuracy: 0.4000
Epoch 4972/5000
 - 0s - loss: 0.2705 - accuracy: 0.9300 - val_loss: 3.5799 - val_accuracy: 0.4000
Epoch 4973/5000
 - 0s - loss: 0.2705 - accuracy: 0.9300 - val_loss: 3.5818 - val_accuracy: 0.4000
Epoch 4974/5000
 - 0s - loss: 0.2705 - accuracy: 0.9300 - val_loss: 3.5837 - val_accuracy: 0.4000
Epoch 4975/5000
 - 0s - loss: 0.2704 - accuracy: 0.9300 - val_loss: 3.5838 - val_accuracy: 0.4000
Epoch 4976/5000
 - 0s - loss: 0.2704 - accuracy: 0.9300 - val_loss: 3.5843 - val_accuracy: 0.4000
Epoch 4977/5000
 - 0s - loss: 0.2704 - accuracy: 0.9300 - val_loss: 3.5847 - val_accuracy: 0.4000
Epoch 4978/5000
 - 0s - loss: 0.2704 - accuracy: 0.9300 - val_loss: 3.5856 - val_accuracy: 0.4000
Epoch 4979/5000
 - 0

In [34]:
max(hist.history['val_accuracy'])

0.5

In [35]:
from keras.models import load_model

b_model = load_model('best_5model.h5')
#b_model = load_model('./10testset_best_model/best_5model.h5')

answer = []
pred = []
answer_per = []
predict_per = []

for i in range(len(test56)) :
    test = test56[i].reshape(1,128)
    label = np.argmax(test_y[i])
    predict = b_model.predict(test)
    max_val = np.argmax(predict[0])
    answer.append(label)
    pred.append(max_val)
    answer_per.append(predict[0][label])
    predict_per.append(predict[0])
    print(label, max_val, predict[0][label], predict[0][max_val])

0 0 0.1402464 0.1402464
9 0 0.13084361 0.1787901
1 1 0.19801801 0.19801801
2 0 0.11992934 0.14744522
3 0 0.087131985 0.1325048
4 8 0.13523947 0.17227396
5 5 0.11341229 0.11341229
6 6 0.19124204 0.19124204
7 8 0.06741823 0.13516186
8 8 0.22258438 0.22258438


# 방법 6. 128 embedding DNN

In [36]:
def good_model():
    # create model
    model = Sequential()
    model.add(Dense(400, input_dim=128, init='normal', activation='relu'))
    model.add(Dense(100, init='normal', activation='relu'))
    model.add(Dense(10, init='normal', activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [37]:
# build the model
model2 = good_model()
mc = ModelCheckpoint('best_6model.h5', monitor='val_accuracy', mode='max', save_best_only=True)
# Fit the model
hist = model2.fit(train56, train_y, validation_data=(test56, test_y), nb_epoch=5000, batch_size=200, verbose=2, callbacks=[mc])
# Final evaluation of the model
scores = model2.evaluate(test56, test_y, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

c:\users\guswl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(400, input_dim=128, activation="relu", kernel_initializer="normal")`
  after removing the cwd from sys.path.
c:\users\guswl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(100, activation="relu", kernel_initializer="normal")`
  """
c:\users\guswl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(10, activation="softmax", kernel_initializer="normal")`
  
c:\users\guswl\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Train on 500 samples, validate on 10 samples
Epoch 1/5000
 - 0s - loss: 2.2995 - accuracy: 0.1680 - val_loss: 2.2960 - val_accuracy: 0.2000
Epoch 2/5000
 - 0s - loss: 2.2832 - accuracy: 0.4200 - val_loss: 2.2883 - val_accuracy: 0.2000
Epoch 3/5000
 - 0s - loss: 2.2628 - accuracy: 0.4580 - val_loss: 2.2770 - val_accuracy: 0.1000
Epoch 4/5000
 - 0s - loss: 2.2350 - accuracy: 0.4640 - val_loss: 2.2609 - val_accuracy: 0.2000
Epoch 5/5000
 - 0s - loss: 2.1971 - accuracy: 0.4580 - val_loss: 2.2387 - val_accuracy: 0.2000
Epoch 6/5000
 - 0s - loss: 2.1481 - accuracy: 0.4600 - val_loss: 2.2106 - val_accuracy: 0.3000
Epoch 7/5000
 - 0s - loss: 2.0871 - accuracy: 0.4660 - val_loss: 2.1740 - val_accuracy: 0.3000
Epoch 8/5000
 - 0s - loss: 2.0125 - accuracy: 0.4900 - val_loss: 2.1264 - val_accuracy: 0.2000
Epoch 9/5000
 - 0s - loss: 1.9243 - accuracy: 0.5260 - val_loss: 2.0715 - val_accuracy: 0.2000
Epoch 10/5000
 - 0s - loss: 1.8300 - accuracy: 0.5300 - val_loss: 2.0136 - val_accuracy: 0.3000
Epoc

Epoch 86/5000
 - 0s - loss: 0.3189 - accuracy: 0.9020 - val_loss: 3.7489 - val_accuracy: 0.3000
Epoch 87/5000
 - 0s - loss: 0.3201 - accuracy: 0.8960 - val_loss: 3.7191 - val_accuracy: 0.4000
Epoch 88/5000
 - 0s - loss: 0.3114 - accuracy: 0.8960 - val_loss: 3.7516 - val_accuracy: 0.4000
Epoch 89/5000
 - 0s - loss: 0.3150 - accuracy: 0.8960 - val_loss: 3.8816 - val_accuracy: 0.4000
Epoch 90/5000
 - 0s - loss: 0.3110 - accuracy: 0.8980 - val_loss: 3.9041 - val_accuracy: 0.4000
Epoch 91/5000
 - 0s - loss: 0.3057 - accuracy: 0.8980 - val_loss: 3.8071 - val_accuracy: 0.3000
Epoch 92/5000
 - 0s - loss: 0.3033 - accuracy: 0.9020 - val_loss: 3.7996 - val_accuracy: 0.4000
Epoch 93/5000
 - 0s - loss: 0.3015 - accuracy: 0.9060 - val_loss: 3.8939 - val_accuracy: 0.4000
Epoch 94/5000
 - 0s - loss: 0.2996 - accuracy: 0.9040 - val_loss: 3.9335 - val_accuracy: 0.4000
Epoch 95/5000
 - 0s - loss: 0.2948 - accuracy: 0.9100 - val_loss: 3.9607 - val_accuracy: 0.4000
Epoch 96/5000
 - 0s - loss: 0.2969 - acc

Epoch 171/5000
 - 0s - loss: 0.1635 - accuracy: 0.9660 - val_loss: 5.5481 - val_accuracy: 0.4000
Epoch 172/5000
 - 0s - loss: 0.1637 - accuracy: 0.9600 - val_loss: 5.5988 - val_accuracy: 0.4000
Epoch 173/5000
 - 0s - loss: 0.1640 - accuracy: 0.9600 - val_loss: 5.5416 - val_accuracy: 0.5000
Epoch 174/5000
 - 0s - loss: 0.1624 - accuracy: 0.9600 - val_loss: 5.5625 - val_accuracy: 0.4000
Epoch 175/5000
 - 0s - loss: 0.1648 - accuracy: 0.9580 - val_loss: 5.6362 - val_accuracy: 0.4000
Epoch 176/5000
 - 0s - loss: 0.1574 - accuracy: 0.9700 - val_loss: 5.6448 - val_accuracy: 0.4000
Epoch 177/5000
 - 0s - loss: 0.1582 - accuracy: 0.9580 - val_loss: 5.6214 - val_accuracy: 0.5000
Epoch 178/5000
 - 0s - loss: 0.1563 - accuracy: 0.9640 - val_loss: 5.7273 - val_accuracy: 0.4000
Epoch 179/5000
 - 0s - loss: 0.1581 - accuracy: 0.9640 - val_loss: 5.8123 - val_accuracy: 0.4000
Epoch 180/5000
 - 0s - loss: 0.1563 - accuracy: 0.9660 - val_loss: 5.6482 - val_accuracy: 0.4000
Epoch 181/5000
 - 0s - loss: 0

Epoch 256/5000
 - 0s - loss: 0.0817 - accuracy: 0.9920 - val_loss: 7.2529 - val_accuracy: 0.4000
Epoch 257/5000
 - 0s - loss: 0.0779 - accuracy: 0.9900 - val_loss: 7.3385 - val_accuracy: 0.5000
Epoch 258/5000
 - 0s - loss: 0.0765 - accuracy: 0.9880 - val_loss: 7.3714 - val_accuracy: 0.4000
Epoch 259/5000
 - 0s - loss: 0.0738 - accuracy: 0.9920 - val_loss: 7.4221 - val_accuracy: 0.4000
Epoch 260/5000
 - 0s - loss: 0.0751 - accuracy: 0.9920 - val_loss: 7.4001 - val_accuracy: 0.5000
Epoch 261/5000
 - 0s - loss: 0.0733 - accuracy: 0.9920 - val_loss: 7.2546 - val_accuracy: 0.5000
Epoch 262/5000
 - 0s - loss: 0.0753 - accuracy: 0.9860 - val_loss: 7.4014 - val_accuracy: 0.5000
Epoch 263/5000
 - 0s - loss: 0.0702 - accuracy: 0.9960 - val_loss: 7.5829 - val_accuracy: 0.4000
Epoch 264/5000
 - 0s - loss: 0.0726 - accuracy: 0.9900 - val_loss: 7.5627 - val_accuracy: 0.4000
Epoch 265/5000
 - 0s - loss: 0.0702 - accuracy: 0.9940 - val_loss: 7.3984 - val_accuracy: 0.5000
Epoch 266/5000
 - 0s - loss: 0

...

Epoch 4614/5000
 - 0s - loss: 9.0551e-07 - accuracy: 1.0000 - val_loss: 28.0965 - val_accuracy: 0.4000
Epoch 4615/5000
 - 0s - loss: 9.0575e-07 - accuracy: 1.0000 - val_loss: 28.0902 - val_accuracy: 0.4000
Epoch 4616/5000
 - 0s - loss: 9.0289e-07 - accuracy: 1.0000 - val_loss: 28.0833 - val_accuracy: 0.4000
Epoch 4617/5000
 - 0s - loss: 9.0170e-07 - accuracy: 1.0000 - val_loss: 28.0875 - val_accuracy: 0.4000
Epoch 4618/5000
 - 0s - loss: 9.0074e-07 - accuracy: 1.0000 - val_loss: 28.0985 - val_accuracy: 0.4000
Epoch 4619/5000
 - 0s - loss: 8.9717e-07 - accuracy: 1.0000 - val_loss: 28.1091 - val_accuracy: 0.4000
Epoch 4620/5000
 - 0s - loss: 8.9645e-07 - accuracy: 1.0000 - val_loss: 28.1180 - val_accuracy: 0.4000
Epoch 4621/5000
 - 0s - loss: 8.9669e-07 - accuracy: 1.0000 - val_loss: 28.1246 - val_accuracy: 0.4000
Epoch 4622/5000
 - 0s - loss: 8.9717e-07 - accuracy: 1.0000 - val_loss: 28.1157 - val_accuracy: 0.4000
Epoch 4623/5000
 - 0s - loss: 8.9240e-07 - accuracy: 1.0000 - val_loss: 2

Epoch 4694/5000
 - 0s - loss: 7.9703e-07 - accuracy: 1.0000 - val_loss: 28.3243 - val_accuracy: 0.4000
Epoch 4695/5000
 - 0s - loss: 7.9346e-07 - accuracy: 1.0000 - val_loss: 28.3327 - val_accuracy: 0.4000
Epoch 4696/5000
 - 0s - loss: 7.9512e-07 - accuracy: 1.0000 - val_loss: 28.3439 - val_accuracy: 0.4000
Epoch 4697/5000
 - 0s - loss: 7.9536e-07 - accuracy: 1.0000 - val_loss: 28.3554 - val_accuracy: 0.4000
Epoch 4698/5000
 - 0s - loss: 7.9465e-07 - accuracy: 1.0000 - val_loss: 28.3499 - val_accuracy: 0.4000
Epoch 4699/5000
 - 0s - loss: 7.9346e-07 - accuracy: 1.0000 - val_loss: 28.3441 - val_accuracy: 0.4000
Epoch 4700/5000
 - 0s - loss: 7.8845e-07 - accuracy: 1.0000 - val_loss: 28.3423 - val_accuracy: 0.4000
Epoch 4701/5000
 - 0s - loss: 7.8845e-07 - accuracy: 1.0000 - val_loss: 28.3417 - val_accuracy: 0.4000
Epoch 4702/5000
 - 0s - loss: 7.8630e-07 - accuracy: 1.0000 - val_loss: 28.3430 - val_accuracy: 0.4000
Epoch 4703/5000
 - 0s - loss: 7.8654e-07 - accuracy: 1.0000 - val_loss: 2

Epoch 4774/5000
 - 0s - loss: 7.0166e-07 - accuracy: 1.0000 - val_loss: 28.5675 - val_accuracy: 0.4000
Epoch 4775/5000
 - 0s - loss: 6.9976e-07 - accuracy: 1.0000 - val_loss: 28.5692 - val_accuracy: 0.4000
Epoch 4776/5000
 - 0s - loss: 6.9952e-07 - accuracy: 1.0000 - val_loss: 28.5714 - val_accuracy: 0.4000
Epoch 4777/5000
 - 0s - loss: 6.9952e-07 - accuracy: 1.0000 - val_loss: 28.5770 - val_accuracy: 0.4000
Epoch 4778/5000
 - 0s - loss: 6.9666e-07 - accuracy: 1.0000 - val_loss: 28.5809 - val_accuracy: 0.4000
Epoch 4779/5000
 - 0s - loss: 6.9761e-07 - accuracy: 1.0000 - val_loss: 28.5842 - val_accuracy: 0.4000
Epoch 4780/5000
 - 0s - loss: 6.9237e-07 - accuracy: 1.0000 - val_loss: 28.5915 - val_accuracy: 0.4000
Epoch 4781/5000
 - 0s - loss: 6.9427e-07 - accuracy: 1.0000 - val_loss: 28.5993 - val_accuracy: 0.4000
Epoch 4782/5000
 - 0s - loss: 6.9404e-07 - accuracy: 1.0000 - val_loss: 28.5977 - val_accuracy: 0.4000
Epoch 4783/5000
 - 0s - loss: 6.9189e-07 - accuracy: 1.0000 - val_loss: 2

Epoch 4854/5000
 - 0s - loss: 6.1989e-07 - accuracy: 1.0000 - val_loss: 28.8172 - val_accuracy: 0.4000
Epoch 4855/5000
 - 0s - loss: 6.1774e-07 - accuracy: 1.0000 - val_loss: 28.8121 - val_accuracy: 0.4000
Epoch 4856/5000
 - 0s - loss: 6.1798e-07 - accuracy: 1.0000 - val_loss: 28.8064 - val_accuracy: 0.4000
Epoch 4857/5000
 - 0s - loss: 6.1703e-07 - accuracy: 1.0000 - val_loss: 28.8053 - val_accuracy: 0.4000
Epoch 4858/5000
 - 0s - loss: 6.1679e-07 - accuracy: 1.0000 - val_loss: 28.8054 - val_accuracy: 0.4000
Epoch 4859/5000
 - 0s - loss: 6.1536e-07 - accuracy: 1.0000 - val_loss: 28.8095 - val_accuracy: 0.4000
Epoch 4860/5000
 - 0s - loss: 6.1560e-07 - accuracy: 1.0000 - val_loss: 28.8180 - val_accuracy: 0.4000
Epoch 4861/5000
 - 0s - loss: 6.1035e-07 - accuracy: 1.0000 - val_loss: 28.8250 - val_accuracy: 0.4000
Epoch 4862/5000
 - 0s - loss: 6.1154e-07 - accuracy: 1.0000 - val_loss: 28.8311 - val_accuracy: 0.4000
Epoch 4863/5000
 - 0s - loss: 6.1035e-07 - accuracy: 1.0000 - val_loss: 2

Epoch 4934/5000
 - 0s - loss: 5.4812e-07 - accuracy: 1.0000 - val_loss: 29.0490 - val_accuracy: 0.4000
Epoch 4935/5000
 - 0s - loss: 5.4431e-07 - accuracy: 1.0000 - val_loss: 29.0429 - val_accuracy: 0.4000
Epoch 4936/5000
 - 0s - loss: 5.4479e-07 - accuracy: 1.0000 - val_loss: 29.0308 - val_accuracy: 0.4000
Epoch 4937/5000
 - 0s - loss: 5.4860e-07 - accuracy: 1.0000 - val_loss: 29.0213 - val_accuracy: 0.4000
Epoch 4938/5000
 - 0s - loss: 5.4812e-07 - accuracy: 1.0000 - val_loss: 29.0287 - val_accuracy: 0.4000
Epoch 4939/5000
 - 0s - loss: 5.4216e-07 - accuracy: 1.0000 - val_loss: 29.0423 - val_accuracy: 0.4000
Epoch 4940/5000
 - 0s - loss: 5.4240e-07 - accuracy: 1.0000 - val_loss: 29.0576 - val_accuracy: 0.4000
Epoch 4941/5000
 - 0s - loss: 5.4192e-07 - accuracy: 1.0000 - val_loss: 29.0732 - val_accuracy: 0.4000
Epoch 4942/5000
 - 0s - loss: 5.4121e-07 - accuracy: 1.0000 - val_loss: 29.0756 - val_accuracy: 0.4000
Epoch 4943/5000
 - 0s - loss: 5.3906e-07 - accuracy: 1.0000 - val_loss: 2

In [38]:
max(hist.history['val_accuracy'])

0.5

In [39]:
from keras.models import load_model

b_model = load_model('best_6model.h5')
#b_model = load_model('./10testset_best_model/best_6model.h5')

answer = []
pred = []
answer_per = []
predict_per = []

for i in range(len(test56)) :
    test = test56[i].reshape(1,128)
    label = np.argmax(test_y[i])
    predict = b_model.predict(test)
    max_val = np.argmax(predict[0])
    answer.append(label)
    pred.append(max_val)
    answer_per.append(predict[0][label])
    predict_per.append(predict[0])
    print(label, max_val, predict[0][label], predict[0][max_val])

0 0 0.65608525 0.65608525
9 0 0.0057953997 0.993856
1 1 0.66437864 0.66437864
2 2 0.53461313 0.53461313
3 8 0.00029897105 0.7986568
4 8 1.6078935e-06 0.9998857
5 6 0.0003694937 0.99953306
6 6 0.99999976 0.99999976
7 8 3.574682e-09 0.96322423
8 8 0.99998415 0.99998415


# 방법 7. SVM

In [40]:
import time
from sklearn.svm import SVC

start = time.time()
clf1 = SVC(C=1, kernel='linear', probability=False)
clf1.fit(train34, train_label)
print("time: ", time.time() - start)

time:  0.17473983764648438


In [41]:
from keras.utils import np_utils

answer = []
pred = []
for i in range(len(test34)) :
    predict = clf1.predict(test34[i].reshape(1,-1))[0]
    #prob = clf.predict_proba(test_x[i].reshape(1,-1))
    prob = clf1.decision_function(test34[i].reshape(1,-1))
    print(test_label[i], predict)
    answer.append(test_label[i])
    pred.append(predict)
    #print(res)
print("Accuracy: {}%".format(clf1.score(test34, test_label) * 100 ))
acc1 = clf1.score(test34, test_label) * 100

0 3
9 9
1 7
2 2
3 3
4 4
5 5
6 6
7 7
8 0
Accuracy: 70.0%


In [42]:
clf2 = SVC(C = 10.0, kernel='rbf', gamma=0.1)

# Train classifier
clf2.fit(train34, train_label)

# Make predictions on unseen test data
clf_predictions = clf2.predict(test34)

In [43]:
answer = []
pred = []
for i in range(len(test34)) :
    predict = clf2.predict(test34[i].reshape(1,-1))[0]
    print(test_label[i], predict)
    answer.append(test_label[i])
    pred.append(predict)
print("Accuracy: {}%".format(clf2.score(test34, test_label) * 100 ))
acc2 = clf2.score(test34, test_label) * 100

0 5
9 5
1 5
2 5
3 5
4 5
5 5
6 5
7 5
8 5
Accuracy: 10.0%


In [44]:
if acc1 >= acc2 :
    joblib.dump(clf1, 'svm.pkl')
    print('save df1')
else :
    joblib.dump(clf2, 'svm.pkl')
    print('save df2')

save df1


In [45]:
from keras.models import load_model
import operator

dict = {'sdr':0, 'phj':1, 'yjy':2, 'jh':3, 'irene':4, 'rjy':5, 'v':6, 'skj':7, 'ysh':8, 'iu':9}
n_dict = {}
for k,v in dict.items() :
    n_dict[v] = k

fin_pred = []
softmax = []  
for i in range(test1.shape[0]) :
    softmax.append(0)
    fin_pred.append('Stranger')

model1 = load_model('best_1model.h5')
model2 = load_model('best_2model.h5')
model3 = load_model('best_3model.h5')
model4 = load_model('best_4model.h5')
model5 = load_model('best_5model.h5')
model6 = load_model('best_6model.h5')
clf = joblib.load('./svm.pkl')

for i in range(len(data)) :
    predict_per = []
    pred_ans = []
    real = []
    
    t1 = test1[i].reshape(1,18)
    t2 = test2[i].reshape(1,136)
    t34 = test34[i].reshape(1,50)
    t56 = test56[i].reshape(1,128)
    
    #predict_per <- softmax, predict_ans <- answer
    predict1 = model1.predict(t1)
    predict_per.append(predict1[0])
    pred_ans.append(np.argmax(predict_per[0]))
    
    predict2 = model2.predict(t2)
    predict_per.append(predict2[0])
    pred_ans.append(np.argmax(predict_per[1]))
    
    predict3 = model3.predict(t34)
    predict_per.append(predict3[0])
    pred_ans.append(np.argmax(predict_per[2]))
    
    predict4 = model4.predict(t34)
    predict_per.append(predict4[0])
    pred_ans.append(np.argmax(predict_per[3]))
    
    predict5 = model5.predict(t56)
    predict_per.append(predict5[0])
    pred_ans.append(np.argmax(predict_per[4]))
    
    predict6 = model6.predict(t56)
    predict_per.append(predict6[0])
    pred_ans.append(np.argmax(predict_per[5]))
    
    predict7 = clf.predict(test34[i].reshape(1,-1))[0]
    pred_ans.append(predict7)
    
    pred_ans = np.array(pred_ans) #해당 인물이 몇번 인물인지
    count = np.bincount(pred_ans)
    val = count.max() # 7개 중 가장 많이 뽑힌 모델이 몇 번 뽑혔는지
    print(np.argmax(test_y[i]), val)
    
    if val > 3 :
        fin_pred[i] = n_dict[np.argmax(count)] # 최종 예측값
    else :
        ans = np.zeros(10)
        for j in range(len(predict_per)) :
            arr = predict_per[j]
            ans = ans + arr
        softmax[i] = ans/6 # softmax 평균, shape:(1,6)
        
        if np.max(softmax[i]) > 0.5 :
            fin_pred[i] = n_dict[val]
            print(np.max(softmax[i]))
        
print('==================== Results ====================')
for i in range(len(fin_pred)) :
    print('actual :', test_label[i],'->', fin_pred[i])
    #print('actual :',n_dict[ans%10],'->', fin_pred[i])

0 3
9 5
1 5
2 5
3 4
4 4
5 4
6 5
7 4
8 5
==================== Results ====================
actual : 0 -> Stranger
actual : 9 -> iu
actual : 1 -> phj
actual : 2 -> yjy
actual : 3 -> jh
actual : 4 -> irene
actual : 5 -> rjy
actual : 6 -> v
actual : 7 -> skj
actual : 8 -> ysh


In [46]:
from keras.models import load_model
import operator

dict = {'sdr':0, 'phj':1, 'yjy':2, 'jh':3, 'irene':4, 'rjy':5, 'v':6, 'skj':7, 'ysh':8, 'iu':9}
n_dict = {}
for k,v in dict.items() :
    n_dict[v] = k

fin_pred = []
softmax = []  
for i in range(test1.shape[0]) :
    softmax.append(0)
    fin_pred.append('Stranger')

model1 = load_model('best_1model.h5')
model2 = load_model('best_2model.h5')
model3 = load_model('best_3model.h5')
model4 = load_model('best_4model.h5')
model5 = load_model('best_5model.h5')
model6 = load_model('best_6model.h5')
clf = joblib.load('./svm.pkl')

for i in range(len(data)) :
    predict_per = []
    pred_ans = []
    real = []
    
    t1 = test1[i].reshape(1,18)
    t2 = test2[i].reshape(1,136)
    t34 = test34[i].reshape(1,50)
    t56 = test56[i].reshape(1,128)
    
    #predict_per <- softmax, predict_ans <- answer
    predict1 = model1.predict(t1)
    predict_per.append(predict1[0])
    pred_ans.append(np.argmax(predict_per[0]))
    
    predict2 = model2.predict(t2)
    predict_per.append(predict2[0])
    pred_ans.append(np.argmax(predict_per[1]))
    
    predict3 = model3.predict(t34)
    predict_per.append(predict3[0])
    pred_ans.append(np.argmax(predict_per[2]))
    
    predict4 = model4.predict(t34)
    predict_per.append(predict4[0])
    pred_ans.append(np.argmax(predict_per[3]))
    
    predict5 = model5.predict(t56)
    predict_per.append(predict5[0])
    pred_ans.append(np.argmax(predict_per[4]))
    
    predict6 = model6.predict(t56)
    predict_per.append(predict6[0])
    pred_ans.append(np.argmax(predict_per[5]))
    
    predict7 = clf.predict(test34[i].reshape(1,-1))[0]
    pred_ans.append(predict7)
    
    pred_ans = np.array(pred_ans) #해당 인물이 몇번 인물인지
    count = np.bincount(pred_ans)
    val = count.max() # 7개 중 가장 많이 뽑힌 모델이 몇 번 뽑혔는지
    print(np.argmax(test_y[i]), val)
    
    if val > 4 :
        fin_pred[i] = n_dict[np.argmax(count)] # 최종 예측값
    else :
        ans = np.zeros(10)
        for j in range(len(predict_per)) :
            arr = predict_per[j]
            ans = ans + arr
        softmax[i] = ans/6 # softmax 평균, shape:(1,6)
        
        if np.max(softmax[i]) > 0.5 :
            fin_pred[i] = n_dict[val]
        if val == 4 :
            num_count = []
            for k in range(len(count)) :
                if count[k] == 4 :
                    num_count.append(k)
            if len(num_count) == 1 :
                fin_pred[i] = n_dict[np.argmax(count)]
                print(fin_pred[i])
            else :
                if pred_ans[6] != 100 :
                    fin_pred[i] = n_dict[pred_ans[6]]
                else :
                    compare_softmax = []
                    for m in num_count :
                        compare_softmax.append(ans[m])
                        print(compare_softmax)
                    compare_softmax = np.array(compare_softmax)
                    fin_pred[i] = n_dict[num_count[np.argmax(compare_softmax)]]
print('==================== Results ====================')
for i in range(len(fin_pred)) :
    print('actual :', test_label[i],'->', fin_pred[i])
    #print('actual :',n_dict[ans%10],'->', fin_pred[i])

0 3
9 5
1 5
2 5
3 4
jh
4 4
irene
5 4
rjy
6 5
7 4
skj
8 5
==================== Results ====================
actual : 0 -> Stranger
actual : 9 -> iu
actual : 1 -> phj
actual : 2 -> yjy
actual : 3 -> jh
actual : 4 -> irene
actual : 5 -> rjy
actual : 6 -> v
actual : 7 -> skj
actual : 8 -> ysh
